In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from umap import UMAP
import warnings
warnings.filterwarnings('ignore')
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
import os
import matplotlib.pyplot as plt
import networkx as nx
try:
    from tqdm.notebook import tqdm  # Jupyter
except ImportError:
    from tqdm import tqdm  # terminal

In [ ]:
CALIBRATION_AVAILABLE = True

# (Function) Data_loader

In [ ]:
class VariantDataLoader:
    """
    Annovar 주석 파일 + STRING PPI 네트워크 로더
    """
    def __init__(self, ppi_source='string'):
        self.ppi_source = ppi_source
        self.gene_to_idx = {}  # 유전자명 → 노드 인덱스 매핑
        self.idx_to_gene = {}  # 노드 인덱스 → 유전자명 매핑
        self.protein_to_gene = {}  # Protein ID → Gene Symbol 매핑
        self.gene_to_protein = {}  # Gene Symbol → Protein ID 매핑
        
    def load_annovar_data(self, annovar_file, label_column='Label'):
        """
        Annovar 주석 파일 로딩
        
        Parameters:
        -----------
        annovar_file: str
            Annovar 주석 파일 경로 (.txt, .csv)
        label_column: str
            Pathogenic/Benign 라벨 컬럼명
            
        Returns:
        --------
        df: pd.DataFrame
        """
        # 파일 로딩
        if annovar_file.endswith('.csv'):
            df = pd.read_csv(annovar_file)
        else:
            df = pd.read_csv(annovar_file, sep='\t')
        
        print(f"Loaded {len(df)} variants from {annovar_file}")
        return df
    
    def preprocess_features(self, df):
        """
        19개 특성 전처리
        
        Parameters:
        -----------
        df: pd.DataFrame
            Annovar 데이터프레임
            
        Returns:
        --------
        X: np.ndarray (n_variants, 19)
        feature_names: list
        """
        feature_columns = [
            'SIFT4G_score', 'LRT_score', 'MutationTaster_score',
            'MutationAssessor_score', 'FATHMM_score', 'PROVEAN_score',
            'CADD_phred', 'integrated_fitCons_score', 'LINSIGHT',
            'GERP++_RS', 'phyloP100way_vertebrate', 'phyloP470way_mammalian',
            'phastCons100way_vertebrate', 'phastCons470way_mammalian',
            'SiPhy_29way_logOdds', 'bStatistic',
            'gnomad41_genome', 'gnomad41_exome', 'AF_patient'
        ]
        
        # 존재하는 컬럼만 선택
        available_features = [col for col in feature_columns if col in df.columns]
        print(f"Using {len(available_features)} features: {available_features}")
        
        X = df[available_features].copy()
        
        # Missing value 처리
        # 1. '.' 를 NaN으로 변환
        X = X.replace('.', np.nan)
        
        # 2. 문자열을 숫자로 변환
        for col in X.columns:
            X[col] = pd.to_numeric(X[col], errors='coerce')
        
        # 3. NaN 처리 전략
        # - 빈도 데이터 (gnomAD, AF): 0으로 채움 (관찰 안 됨 = 희귀)
        freq_cols = ['gnomad41_genome', 'gnomad41_exome', 'AF']
        for col in freq_cols:
            if col in X.columns:
                X[col].fillna(0, inplace=True)
        
        # - 스코어 데이터: median으로 채움
        score_cols = [col for col in X.columns if col not in freq_cols]
        for col in score_cols:
            if col in X.columns:
                X[col].fillna(X[col].median(), inplace=True)
        
        # 4. 여전히 NaN이 남아있으면 0으로
        X.fillna(0, inplace=True)
        
        print(f"Feature shape: {X.shape}")
        print(f"Missing values: {X.isna().sum().sum()}")
        
        return X.values, available_features
    
    def extract_labels(self, df, label_column='ClinVar_SIG', include_vus=False):
        """
        병원성 라벨 추출 (학습용)
        
        Parameters:
        -----------
        df: pd.DataFrame
        label_column: str
            병원성 라벨 컬럼명
        include_vus: bool
            VUS를 학습 데이터에 포함할지 (기본 False)
            
        Returns:
        --------
        y_binary: np.ndarray (n_labeled_variants,)
            1 = Pathogenic, 0 = Benign
        labeled_idx: np.ndarray
            라벨이 있는 변이의 인덱스 (VUS 제외)
        """
        if label_column not in df.columns:
            raise ValueError(f"Label column '{label_column}' not found in dataframe")
        
        # 라벨 매핑
        pathogenic_keywords = ['pathogenic', 'likely_pathogenic', 'Pathogenic', 'Likely_pathogenic', 'P', 'Pathogenic/Likely_pathogenic']
        benign_keywords = ['benign', 'likely_benign', 'Benign', 'Likely_benign', 'B', 'Benign/Likely_benign']
        vus_keywords = ['uncertain', 'VUS', 'uncertain_significance', 'Uncertain_significance', '.', 'Conflicting_classifications_of_pathogenicity']
        
        label_str = df[label_column].astype(str).str.lower()
        
        # 각 카테고리 구분
        is_pathogenic = label_str.isin([k.lower() for k in pathogenic_keywords])
        is_benign = label_str.isin([k.lower() for k in benign_keywords])
        is_vus = label_str.isin([k.lower() for k in vus_keywords])
        
        # 라벨이 있는 변이만 (Pathogenic or Benign)
        has_label = is_pathogenic | is_benign
        labeled_idx = np.where(has_label)[0]
        
        # 이진 라벨 생성
        y_binary = np.zeros(len(df))
        y_binary[is_pathogenic] = 1
        y_binary[is_benign] = 0
        
        print(f"\n=== Label Statistics ===")
        print(f"Pathogenic: {is_pathogenic.sum()}")
        print(f"Benign: {is_benign.sum()}")
        print(f"VUS (unlabeled): {is_vus.sum()}")
        print(f"Unknown: {(~(is_pathogenic | is_benign | is_vus)).sum()}")
        print(f"Total labeled (train): {len(labeled_idx)}")
        
        if include_vus:
            print("Note: Including VUS in dataset but they won't be used in training")
            return y_binary, None
        else:
            print(f"Excluding VUS from training set")
            return y_binary[labeled_idx], labeled_idx
    
    def extract_gene_mapping(self, df, gene_column='Gene.refGene'):
        """
        Variant → Gene 매핑 추출
        여러 유전자(A;B;C)가 있으면 explode하여 모두 반영
        
        Parameters:
        -----------
        df: pd.DataFrame
        gene_column: str
            유전자 컬럼명 (Gene.refGene, Gene.ensGene 등)
            
        Returns:
        --------
        gene_list: list of lists
            각 변이에 해당하는 유전자명 리스트 (복수 가능)
        """
        if gene_column not in df.columns:
            raise ValueError(f"Gene column '{gene_column}' not found")
        
        gene_list = []
        
        for gene_str in df[gene_column]:
            # Annovar는 missing을 '.'으로 표시
            if pd.isna(gene_str) or gene_str == '.' or gene_str == '' or str(gene_str).strip() == '.':
                # Intergenic variant 처리
                gene_list.append(['UNKNOWN'])
            elif ';' in str(gene_str) or '|' in str(gene_str):
                # 여러 유전자 → 모두 사용 (explode)
                # 세미콜론(;)과 파이프(|) 모두 구분자로 처리
                gene_str_clean = str(gene_str).replace('|', ';')  # 통일
                genes = [g.strip() for g in gene_str_clean.split(';') if g.strip() and g.strip() != '.']
                gene_list.append(genes if genes else ['UNKNOWN'])
            else:
                gene = str(gene_str).strip()
                gene_list.append([gene] if gene and gene != '.' else ['UNKNOWN'])
        
        # 고유 유전자 통계
        all_genes_flat = [g for genes in gene_list for g in genes]
        unique_genes = set(all_genes_flat) - {'UNKNOWN'}
        
        print(f"Extracted {len(unique_genes)} unique genes from {len(gene_list)} variants")
        print(f"  Multi-gene variants: {sum([len(g) > 1 for g in gene_list])}")
        print(f"  Unknown genes: {all_genes_flat.count('UNKNOWN')}")
        
        if len(unique_genes) == 0:
            print("⚠ WARNING: No valid genes found! All values are '.' (missing)")
            print(f"Sample values in {gene_column}: {df[gene_column].value_counts().head()}")
        
        return gene_list
    
    def load_string_with_gene_filter(self, string_file, alias_file, my_genes, 
                                      mm_seed_genes=None, score_threshold=400, organism='9606'):
        """
        유전자 리스트 기반으로 PPI 네트워크 필터링 로딩 (효율적)
        MM seed 유전자는 PPI에 없어도 강제로 추가!
        
        Parameters:
        -----------
        string_file: str
            STRING PPI 파일 경로
        alias_file: str
            STRING alias 파일 경로
        my_genes: list
            내 데이터에 있는 유전자 리스트
        mm_seed_genes: list
            MM 허브 유전자 리스트 (RWR seed용, PPI에 강제 추가)
        score_threshold: int
            Combined score 임계값
        organism: str
            생물 종 ID (9606 = human)
            
        Returns:
        --------
        G: networkx.Graph
            PPI 네트워크 (데이터 유전자 + MM seed)
        """
        print(f"\n=== Loading PPI with gene filtering ===")
        print(f"Target genes: {len(my_genes)}")
        if mm_seed_genes:
            print(f"MM seed genes (will be added if missing): {len(mm_seed_genes)}")
        
        # 데이터 유전자 + MM seed 통합
        all_genes_to_load = set(my_genes)
        if mm_seed_genes:
            all_genes_to_load.update(mm_seed_genes)
        
        # 1. Alias 매핑
        print("\n[1/3] Loading alias mappings...")
        alias_df = pd.read_csv(
            alias_file,
            sep="\t", 
            header=None,
            names=["STRING_id", "alias", "source"]
        )
        
        # 내 유전자 + MM 유전자 매칭
        gene_map = alias_df[alias_df["alias"].isin(all_genes_to_load)].drop_duplicates("alias")
        mapped_genes = gene_map["alias"].nunique()
        print(f"✓ Mapped {mapped_genes}/{len(all_genes_to_load)} genes to STRING IDs")
        
        # STRING_id → Gene symbol dict
        string_to_gene = dict(zip(gene_map["STRING_id"], gene_map["alias"]))
        self.protein_to_gene = string_to_gene
        self.gene_to_protein = {v: k for k, v in string_to_gene.items()}
        
        # 2. PPI 로딩 (필터링)
        print("\n[2/3] Loading PPI network...")
        mapped_ids = gene_map["STRING_id"].tolist()
        
        # 청크로 읽으면서 필터링
        edges = []
        chunk_size = 100000
        
        with tqdm(desc="Filtering PPI edges", unit=" edges") as pbar:
            for chunk in pd.read_csv(string_file, sep=r'\s+', chunksize=chunk_size, engine='python'):
                # 내 유전자 + MM 유전자만
                chunk_filtered = chunk[
                    (chunk['protein1'].isin(mapped_ids)) &
                    (chunk['protein2'].isin(mapped_ids))
                ]
                
                # Score 필터링
                if 'combined_score' in chunk_filtered.columns:
                    chunk_filtered = chunk_filtered[chunk_filtered['combined_score'] >= score_threshold]
                
                # Edge 추출 (Gene Symbol로 변환)
                for _, row in chunk_filtered.iterrows():
                    p1 = string_to_gene.get(row['protein1'], row['protein1'])
                    p2 = string_to_gene.get(row['protein2'], row['protein2'])
                    score = float(row.get('combined_score', 1000)) / 1000.0
                    edges.append((p1, p2, score))
                
                pbar.update(len(chunk))
        
        print(f"✓ Found {len(edges)} edges")
        
        # 3. NetworkX 그래프 생성
        print("\n[3/3] Building graph...")
        G = nx.Graph()
        G.add_weighted_edges_from(edges)
        
        # 4. MM seed 유전자 강제 추가 (PPI에 없는 경우)
        if mm_seed_genes:
            added_mm = []
            for mm_gene in mm_seed_genes:
                if mm_gene not in G.nodes():
                    G.add_node(mm_gene)
                    added_mm.append(mm_gene)
            
            if added_mm:
                print(f"\n⚠ {len(added_mm)} MM genes NOT in STRING PPI:")
                print(f"  {', '.join(added_mm[:10])}{'...' if len(added_mm) > 10 else ''}")
                print(f"  → Added as isolated nodes (will still work as RWR seeds)")
        
        print(f"\n✓ Final PPI network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
        
        return G
    
    def create_prediction_labels(self, df):
        """
        예측 스코어 기반으로 라벨 생성 (ClinVar 없을 때)
        
        Parameters:
        -----------
        df: pd.DataFrame
        
        Returns:
        --------
        y_binary: np.ndarray
        labeled_idx: np.ndarray
        """
        print("\n=== Creating labels from prediction scores ===")
        
        # CADD score 기반 (>20 = pathogenic)
        cadd = pd.to_numeric(df['CADD_phred'].replace('.', np.nan), errors='coerce')
        
        # 여러 예측 스코어 종합
        votes = pd.DataFrame({
            'CADD': cadd > 20,
            'SIFT': df['SIFT4G_pred'] == 'D',
            'MutTaster': df['MutationTaster_pred'] == 'D'
        })
        
        # 2개 이상 pathogenic 예측 → 1
        vote_count = votes.sum(axis=1)
        y_binary = (vote_count >= 2).astype(int)
        
        # 최소 1개 스코어라도 있는 것만
        has_score = ~cadd.isna()
        labeled_idx = np.where(has_score)[0]
        
        print(f"Labeled by prediction: {len(labeled_idx)}")
        print(f"  Pathogenic (>=2 votes): {(y_binary[labeled_idx] == 1).sum()}")
        print(f"  Benign (<2 votes): {(y_binary[labeled_idx] == 0).sum()}")
        
        return y_binary.values[labeled_idx], labeled_idx
    
    def load_string_ppi(self, string_file, alias_file=None, score_threshold=400, organism='9606'):
        """
        STRING PPI 네트워크 로딩 (메모리 효율적)
        
        Parameters:
        -----------
        string_file: str
            STRING 파일 경로 (protein.links.v12.0.txt 형식)
        score_threshold: int
            Combined score 임계값 (0-1000, 기본 400 = medium confidence)
        organism: str
            생물 종 ID (9606 = human)
            
        Returns:
        --------
        G: networkx.Graph
            PPI 네트워크
        """
        print(f"Loading STRING PPI network from {string_file}...")
        
        # 청크 단위로 읽기 (메모리 절약)
        chunk_size = 100000
        edges = []
        
        try:
            # 첫 줄로 컬럼명 확인
            first_line = pd.read_csv(string_file, sep=' ', nrows=1)
            print(f"Detected columns: {list(first_line.columns)}")
            
            # 청크 단위로 읽기
            for chunk in pd.read_csv(string_file, sep=' ', chunksize=chunk_size, 
                                     dtype={'protein1': str, 'protein2': str, 'combined_score': int}):
                
                # Score 필터링
                if 'combined_score' in chunk.columns:
                    chunk = chunk[chunk['combined_score'] >= score_threshold]
                
                # 유전자 심볼 추출 (9606.ENSP00000XXX → gene symbol 또는 protein ID)
                protein1_col = chunk.columns[0]
                protein2_col = chunk.columns[1]
                
                for _, row in chunk.iterrows():
                    # organism prefix 제거
                    p1 = str(row[protein1_col]).replace(f'{organism}.', '')
                    p2 = str(row[protein2_col]).replace(f'{organism}.', '')
                    
                    score = row.get('combined_score', 1000) if 'combined_score' in chunk.columns else 1000
                    edges.append((p1, p2, score / 1000.0))
                
                print(f"  Processed {len(edges)} edges...", end='\r')
            
            print(f"\nTotal edges after filtering: {len(edges)}")
            
        except Exception as e:
            print(f"Error reading file: {e}")
            print("Attempting alternative reading method...")
            
            # 대안: 직접 파일 읽기
            edges = []
            with open(string_file, 'r') as f:
                header = f.readline()  # 헤더 건너뛰기
                
                # 파일 크기로 진행률 추정
                import os
                file_size = os.path.getsize(string_file)
                
                with tqdm(total=file_size, desc="Reading PPI file", unit="B", unit_scale=True) as pbar:
                    for i, line in enumerate(f):
                        parts = line.strip().split()
                        if len(parts) >= 3:
                            try:
                                score = float(parts[2])  # 명시적으로 float 변환
                                if score >= score_threshold:
                                    p1 = parts[0].replace(f'{organism}.', '')
                                    p2 = parts[1].replace(f'{organism}.', '')
                                    edges.append((p1, p2, score / 1000.0))
                            except (ValueError, IndexError):
                                continue
                        
                        if i % 10000 == 0:
                            pbar.update(f.tell() - pbar.n)
            
            print(f"\nTotal edges: {len(edges)}")
        
        # NetworkX 그래프 생성
        G = nx.Graph()
        G.add_weighted_edges_from(edges)
        
        print(f"Created PPI network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
        
        return G
    
    def create_gene_to_node_mapping(self, gene_list, ppi_graph):
        """
        유전자명 → 노드 인덱스 매핑 생성
        gene_list가 list of lists일 경우 첫 번째 유전자 사용
        (여러 유전자가 있으면 첫 번째를 대표로)
        
        Parameters:
        -----------
        gene_list: list of lists or list
            변이별 유전자명 리스트
        ppi_graph: networkx.Graph
            PPI 네트워크
            
        Returns:
        --------
        node_indices: np.ndarray
            각 변이에 대응하는 노드 인덱스
        """
        # PPI 네트워크의 모든 유전자
        all_genes = list(ppi_graph.nodes())
        
        # 유전자명 → 인덱스 매핑
        self.gene_to_idx = {gene: idx for idx, gene in enumerate(all_genes)}
        self.idx_to_gene = {idx: gene for gene, idx in self.gene_to_idx.items()}
        
        # UNKNOWN 노드 추가 (PPI에 없는 유전자용)
        unknown_idx = len(all_genes)
        self.gene_to_idx['UNKNOWN'] = unknown_idx
        self.idx_to_gene[unknown_idx] = 'UNKNOWN'
        
        # 각 변이의 노드 인덱스
        node_indices = []
        for genes in gene_list:
            # list of lists인 경우 첫 번째 유전자 사용
            if isinstance(genes, list):
                gene = genes[0]
            else:
                gene = genes
            
            if gene in self.gene_to_idx:
                node_indices.append(self.gene_to_idx[gene])
            else:
                node_indices.append(unknown_idx)
        
        print(f"Mapped {len(node_indices)} variants to {len(set(node_indices))} unique nodes")
        
        return np.array(node_indices)
    
    
    def compute_rwr_from_seeds(self, ppi_graph, seed_genes, restart_prob=0.15, max_iter=100,
                              remove_super_hubs=True, super_hub_genes=None):
        """
        MM 허브 유전자로부터 Random Walk with Restart (RWR) 계산
        직접 행렬 연산 구현으로 세밀한 제어 가능
        
        Parameters:
        -----------
        ppi_graph: networkx.Graph
        seed_genes: list
            MM 관련 허브 유전자 리스트
        restart_prob: float
            재시작 확률 (기본 0.15)
        max_iter: int
            최대 반복 횟수
        remove_super_hubs: bool
            슈퍼 허브(UBC, Histone 등) 제거 여부 (기본 True)
        super_hub_genes: list
            제거할 슈퍼 허브 리스트 (None이면 기본 리스트 사용)
            
        Returns:
        --------
        rwr_scores: dict
            {gene: rwr_score}
        """
        print(f"\n=== Computing RWR from {len(seed_genes)} seed genes ===")
        
        # 슈퍼 허브 리스트 (기본값)
        if super_hub_genes is None:
            super_hub_genes = [
                # UBC (Ubiquitin) 계열
                'UBC', 'UBB', 'UBA52', 'RPS27A',
                # Histone 계열
                'HIST1H1A', 'HIST1H1B', 'HIST1H1C', 'HIST1H1D', 'HIST1H1E',
                'HIST1H2AA', 'HIST1H2AB', 'HIST1H2AC', 'HIST1H2AD',
                'HIST1H3A', 'HIST1H3B', 'HIST1H3C', 'HIST1H4A',
                'HIST2H2AA', 'HIST2H2AB', 'HIST2H3A',
            ]
        
        # 노드 리스트
        all_nodes = list(ppi_graph.nodes())
        n_nodes = len(all_nodes)
        node_to_idx = {node: i for i, node in enumerate(all_nodes)}
        
        # Seed 유전자 확인
        valid_seeds = [g for g in seed_genes if g in node_to_idx]
        missing_seeds = [g for g in seed_genes if g not in node_to_idx]
        
        print(f"Valid seed genes in PPI: {len(valid_seeds)}/{len(seed_genes)}")
        
        if missing_seeds:
            print(f"⚠ WARNING: {len(missing_seeds)} seeds NOT in PPI graph:")
            print(f"  {', '.join(missing_seeds)}")
        
        if len(valid_seeds) == 0:
            print("❌ ERROR: No seed genes found in PPI network!")
            return {node: 0.0 for node in all_nodes}
        
        # 인접 행렬 (가중치 포함)
        adj_matrix = nx.to_numpy_array(ppi_graph, nodelist=all_nodes, weight='weight')
        
        # 전이 행렬 (열 정규화)
        col_sum = adj_matrix.sum(axis=0)
        col_sum[col_sum == 0] = 1  # Isolated node 처리
        P = adj_matrix / col_sum
        
        # 초기 벡터 (seed genes에 균등 분배)
        r0 = np.zeros(n_nodes)
        for seed in valid_seeds:
            r0[node_to_idx[seed]] = 1.0 / len(valid_seeds)
        
        # RWR 반복
        r = r0.copy()
        for iteration in range(max_iter):
            r_new = (1 - restart_prob) * P.dot(r) + restart_prob * r0
            
            # 수렴 체크
            if np.allclose(r, r_new, atol=1e-6):
                print(f"✓ RWR converged at iteration {iteration+1}")
                break
            r = r_new
        
        # 결과를 dict로 변환
        rwr_scores = {all_nodes[i]: r[i] for i in range(n_nodes)}
        
        print(f"RWR scores range: [{min(rwr_scores.values()):.6f}, {max(rwr_scores.values()):.6f}]")
        
        # 슈퍼 허브 제거 (옵션)
        if remove_super_hubs:
            removed_count = 0
            removed_genes = []
            
            for gene in rwr_scores.keys():
                if gene in super_hub_genes:
                    if rwr_scores[gene] > 0:
                        removed_genes.append((gene, rwr_scores[gene]))
                        rwr_scores[gene] = 0.0
                        removed_count += 1
                elif (gene.startswith('HIST') or 
                      gene.startswith('RPL') or 
                      gene.startswith('RPS')):
                    if rwr_scores[gene] > 0:
                        removed_genes.append((gene, rwr_scores[gene]))
                        rwr_scores[gene] = 0.0
                        removed_count += 1
            
            if removed_count > 0:
                print(f"✓ Removed {removed_count} super-hub genes from RWR scores")
                
                if removed_genes:
                    removed_genes.sort(key=lambda x: x[1], reverse=True)
                    top_removed = removed_genes[:5]
                    print(f"  Top removed super-hubs:")
                    for gene, score in top_removed:
                        print(f"    {gene}: {score:.6f} → 0.0")
        
        return rwr_scores
    
    def compute_ppr_from_seeds(self, ppi_graph, seed_genes, alpha=0.15, max_iter=100,
                              remove_super_hubs=True, super_hub_genes=None):
        """
        MM 허브 유전자로부터 Personalized PageRank (PPR) 계산
        
        PPR은 RWR과 알고리즘적으로 동일하지만, NetworkX의 pagerank를 사용하여
        더 효율적이고 안정적으로 구현됩니다.
        
        Parameters:
        -----------
        ppi_graph: networkx.Graph
        seed_genes: list
            MM 관련 허브 유전자 리스트
        alpha: float
            텔레포트 확률 (personalization으로 돌아갈 확률, 기본 0.15)
            RWR의 restart_prob와 동일
        max_iter: int
            최대 반복 횟수
        remove_super_hubs: bool
            슈퍼 허브(UBC, Histone 등) 제거 여부 (기본 True)
        super_hub_genes: list
            제거할 슈퍼 허브 리스트 (None이면 기본 리스트 사용)
            
        Returns:
        --------
        ppr_scores: dict
            {gene: ppr_score}
        """
        print(f"\n=== Computing PPR from {len(seed_genes)} seed genes ===")
        
        # 슈퍼 허브 리스트 (기본값)
        if super_hub_genes is None:
            super_hub_genes = [
                # UBC (Ubiquitin) 계열
                'UBC', 'UBB', 'UBA52', 'RPS27A',
                # Histone 계열
                'HIST1H1A', 'HIST1H1B', 'HIST1H1C', 'HIST1H1D', 'HIST1H1E',
                'HIST1H2AA', 'HIST1H2AB', 'HIST1H2AC', 'HIST1H2AD',
                'HIST1H3A', 'HIST1H3B', 'HIST1H3C', 'HIST1H4A',
                'HIST2H2AA', 'HIST2H2AB', 'HIST2H3A',
                # Ribosomal proteins (선택적)
                # 'RPL', 'RPS'로 시작하는 것들은 나중에 prefix로 필터링
            ]
        
        # 노드 리스트
        all_nodes = list(ppi_graph.nodes())
        
        # Seed 유전자 확인
        valid_seeds = [g for g in seed_genes if g in ppi_graph.nodes()]
        missing_seeds = [g for g in seed_genes if g not in ppi_graph.nodes()]
        
        print(f"Valid seed genes in PPI: {len(valid_seeds)}/{len(seed_genes)}")
        
        if missing_seeds:
            print(f"⚠ WARNING: {len(missing_seeds)} seeds NOT in PPI graph:")
            print(f"  {', '.join(missing_seeds)}")
        
        if len(valid_seeds) == 0:
            print("❌ ERROR: No seed genes found in PPI network!")
            print("   Cannot compute PPR. Returning zero scores.")
            return {node: 0.0 for node in all_nodes}
        
        # Personalization vector 생성 (seed에만 균등 분배)
        personalization = {node: 0.0 for node in all_nodes}
        seed_weight = 1.0 / len(valid_seeds)
        for seed in valid_seeds:
            personalization[seed] = seed_weight
        
        # NetworkX PageRank with Personalization = PPR
        try:
            ppr_scores = nx.pagerank(
                ppi_graph,
                alpha=alpha,  # teleportation probability (= RWR restart_prob)
                personalization=personalization,
                max_iter=max_iter,
                tol=1e-6,
                weight='weight'  # 가중치 엣지 사용
            )
            print(f"✓ PPR converged successfully")
        except nx.PowerIterationFailedConvergence:
            print("⚠ Warning: PPR did not converge within max_iter, using partial results")
            ppr_scores = nx.pagerank(
                ppi_graph, 
                alpha=alpha, 
                personalization=personalization,
                max_iter=max_iter * 2,  # 더 많은 반복 시도
                weight='weight'
            )
        
        print(f"PPR scores range: [{min(ppr_scores.values()):.6f}, {max(ppr_scores.values()):.6f}]")
        
        # 🔥 슈퍼 허브 제거 (질병 특이성 없는 일반적 허브)
        if remove_super_hubs:
            removed_count = 0
            removed_genes = []
            
            for gene in ppr_scores.keys():
                # 직접 매칭
                if gene in super_hub_genes:
                    if ppr_scores[gene] > 0:
                        removed_genes.append((gene, ppr_scores[gene]))
                        ppr_scores[gene] = 0.0
                        removed_count += 1
                # Prefix 매칭 (Histone, Ribosomal)
                elif (gene.startswith('HIST') or 
                      gene.startswith('RPL') or 
                      gene.startswith('RPS')):
                    if ppr_scores[gene] > 0:
                        removed_genes.append((gene, ppr_scores[gene]))
                        ppr_scores[gene] = 0.0
                        removed_count += 1
            
            if removed_count > 0:
                print(f"✓ Removed {removed_count} super-hub genes from PPR scores")
                print(f"  (UBC, Histone, Ribosomal proteins - general cellular functions)")
                
                # 제거된 것 중 높은 점수였던 것 출력
                if removed_genes:
                    removed_genes.sort(key=lambda x: x[1], reverse=True)
                    top_removed = removed_genes[:5]
                    print(f"  Top removed super-hubs:")
                    for gene, score in top_removed:
                        print(f"    {gene}: {score:.6f} → 0.0")
        
        return ppr_scores
    
    def create_node_features_with_rwr_and_ppr(self, ppi_graph, rwr_scores, ppr_scores, feature_type='centrality+rwr+ppr'):
        """
        노드 특성 생성 (RWR + PPR 스코어 모두 포함)
        
        RWR과 PPR을 둘 다 사용하는 이유:
        - RWR: 직접 구현으로 세밀한 제어
        - PPR: NetworkX 최적화로 안정성
        - 두 점수가 일치하면 신뢰도 ↑, 다르면 추가 정보 제공
        
        Parameters:
        -----------
        ppi_graph: networkx.Graph
        rwr_scores: dict
            {gene: rwr_score}
        ppr_scores: dict
            {gene: ppr_score}
        feature_type: str
            'centrality+rwr+ppr': 중심성 + RWR + PPR 스코어
            
        Returns:
        --------
        node_features: np.ndarray (n_nodes+1, 8)
            [degree_cent, betweenness_cent, closeness_cent, eigenvector_cent, 
             clustering, degree, RWR_score, PPR_score]
        """
        n_nodes = ppi_graph.number_of_nodes()
        
        print("\n=== Computing node features (with RWR + PPR) ===")
        
        # 그래프 중심성 계산
        print("Computing centrality metrics...")
        degree_cent = nx.degree_centrality(ppi_graph)
        betweenness_cent = nx.betweenness_centrality(ppi_graph, k=min(100, n_nodes))
        closeness_cent = nx.closeness_centrality(ppi_graph)
        
        try:
            eigenvector_cent = nx.eigenvector_centrality(ppi_graph, max_iter=1000)
        except:
            print("Warning: Eigenvector centrality failed, using degree centrality instead")
            eigenvector_cent = degree_cent
        
        clustering = nx.clustering(ppi_graph)
        
        # 특성 벡터 구성 (RWR + PPR 둘 다!)
        features = []
        for node in ppi_graph.nodes():
            features.append([
                degree_cent[node],
                betweenness_cent[node],
                closeness_cent[node],
                eigenvector_cent[node],
                clustering[node],
                ppi_graph.degree(node),  # raw degree
                rwr_scores.get(node, 0.0),  # RWR score (MM 관련성 - 직접 구현)
                ppr_scores.get(node, 0.0)   # PPR score (MM 관련성 - NetworkX 구현)
            ])
        
        node_features = np.array(features)
        
        # UNKNOWN 노드 특성 (중립값 = 0으로 설정)
        unknown_features = np.zeros((1, node_features.shape[1]))
        unknown_features[0, -2] = 0.0  # RWR score = 0
        unknown_features[0, -1] = 0.0  # PPR score = 0
        node_features = np.vstack([node_features, unknown_features])
        
        print(f"Node features shape: {node_features.shape}")
        print(f"Features (8D): degree_cent, betweenness_cent, closeness_cent, eigenvector_cent, clustering, degree, RWR_score, PPR_score")
        print(f"✓ Using both RWR and PPR for robustness (complementary signals)")
        print(f"✓ UNKNOWN node features set to neutral (0.0) to avoid bias")
        
        return node_features
    
    def create_adjacency_matrix(self, ppi_graph):
        """
        인접 행렬 생성
        
        Returns:
        --------
        adj_matrix: np.ndarray (n_nodes+1, n_nodes+1)
            마지막 행/열은 UNKNOWN 노드 (자기 자신만 연결)
        """
        n_nodes = ppi_graph.number_of_nodes()
        
        # NetworkX에서 인접 행렬 추출
        adj_matrix = nx.to_numpy_array(ppi_graph)
        
        # UNKNOWN 노드 추가 (자기 자신만 연결)
        unknown_row = np.zeros((1, n_nodes))
        adj_matrix = np.vstack([adj_matrix, unknown_row])
        
        unknown_col = np.zeros((n_nodes + 1, 1))
        unknown_col[-1, -1] = 1.0  # self-loop
        adj_matrix = np.hstack([adj_matrix, unknown_col])
        
        print(f"Adjacency matrix shape: {adj_matrix.shape}")
        
        return adj_matrix

def prepare_variant_data(annovar_file, string_file, mm_genes, 
                        label_column='CLNSIG', gene_column='Gene.refGene',
                        alias_file=None,
                        rwr_restart_prob=0.15, ppr_alpha=0.15,
                        use_prediction_labels=False,
                        exonic_only=True, remove_super_hubs=True):
    """
    전체 데이터 준비 파이프라인
    
    Parameters:
    -----------
    annovar_file: str
        Annovar 주석 파일 경로
    string_file: str
        STRING PPI 파일 경로
    mm_genes: list
        MM 관련 허브 유전자 리스트 (RWR/PPR seed, PPI에 강제 추가됨)
    label_column: str
        병원성 라벨 컬럼명 (기본: 'CLNSIG')
    gene_column: str
        유전자 컬럼명
    alias_file: str
        STRING alias 파일 경로 (None이면 자동 추론)
    rwr_restart_prob: float
        RWR 재시작 확률 (기본 0.15, 직접 구현)
    ppr_alpha: float
        PPR 텔레포트 확률 (기본 0.15, NetworkX 구현)
    remove_super_hubs: bool
        슈퍼 허브(UBC, Histone 등) 점수 제거 여부 (기본: True)
    use_prediction_labels: bool
        ClinVar 없을 때 예측 스코어로 라벨 생성 여부
    exonic_only: bool
        Exonic variant만 사용 여부 (기본 True)
        
    Returns:
    --------
    X_train, y_train, X_all, y_all, node_features, adjacency_matrix,
    node_indices_train, node_indices_all, df_all, loader
    """
    
    loader = VariantDataLoader()
    
    print("="*70)
    print("MM Pathogenic Variant Prediction - Data Preparation")
    print("="*70)
    
    # 1. Annovar 데이터 로딩
    df_all = loader.load_annovar_data(annovar_file, label_column)
    
    # 1.5. Exonic+Splicing 필터링, Synonymous SNV 제외 (옵션)
    if exonic_only and 'Func.refGene' in df_all.columns:
        before = len(df_all)
        
        # Exonic OR Splicing 변이 포함
        functional_mask = (
            df_all['Func.refGene'].str.contains('exonic', case=False, na=False) |
            df_all['Func.refGene'].str.contains('splicing', case=False, na=False)
        )
        df_all = df_all[functional_mask].reset_index(drop=True)
        
        after_func = len(df_all)
        print(f"\n✓ Filtered to exonic/splicing variants: {after_func} (from {before:,})")
        
        # Synonymous SNV 제외 (ExonicFunc.refGene 컬럼 사용)
        if 'ExonicFunc.refGene' in df_all.columns:
            before_syn = len(df_all)
            syn_mask = df_all['ExonicFunc.refGene'].str.contains('synonymous SNV', case=False, na=False)
            df_all = df_all[~syn_mask].reset_index(drop=True)
            
            print(f"✓ Excluded synonymous SNV: {len(df_all)} (removed {before_syn - len(df_all)})")
        
        print(f"  Total filtering rate: {len(df_all)/before*100:.1f}%")
    
    # 2. 특성 추출 (전체)
    X_all, feature_names = loader.preprocess_features(df_all)
    
    # 3. 라벨 추출
    if use_prediction_labels:
        print("\n⚠ Using prediction scores as labels (no ClinVar data)")
        # CADD score 기반 라벨 생성
        y_all_temp, labeled_idx = loader.create_prediction_labels(df_all)
    else:
        # ClinVar 라벨 사용
        y_all_temp, labeled_idx = loader.extract_labels(df_all, label_column, include_vus=False)
    
    # y_all 생성 (전체 데이터에 대해 -1 채우기)
    y_all = np.full(len(df_all), -1)
    if labeled_idx is not None and len(labeled_idx) > 0:
        y_all[labeled_idx] = y_all_temp
        
        # 학습용 데이터
        y_train = y_all_temp
        X_train = X_all[labeled_idx]
        df_train = df_all.iloc[labeled_idx].reset_index(drop=True)
    else:
        print("\n❌ ERROR: No training data available!")
        y_train = np.array([])
        X_train = np.array([]).reshape(0, X_all.shape[1])
        df_train = df_all.iloc[:0]
    
    # 4. Gene 매핑 (전체 & 학습용)
    gene_list_all = loader.extract_gene_mapping(df_all, gene_column)
    gene_list_train = loader.extract_gene_mapping(df_train, gene_column) if len(df_train) > 0 else []
    
    # 고유 유전자 리스트 (UNKNOWN 제외)
    # gene_list_all은 이제 list of lists
    all_genes_flat = [g for genes in gene_list_all for g in genes]
    unique_genes = set(all_genes_flat) - {'UNKNOWN'}
    
    print(f"\nUnique genes in data: {len(unique_genes)}")
    print(f"MM seed genes: {len(mm_genes)}")
    
    # 5. PPI 네트워크 로딩 (데이터 유전자 + MM seed 강제 추가)
    if alias_file:
        ppi_graph = loader.load_string_with_gene_filter(
            string_file=string_file,
            alias_file=alias_file,
            my_genes=list(unique_genes),
            mm_seed_genes=mm_genes,  # 👈 MM 유전자를 PPI에 강제 추가!
            score_threshold=400
        )
    else:
        print("\n⚠ WARNING: No alias file provided. Using unfiltered PPI (may be slow).")
        ppi_graph = loader.load_string_ppi(string_file, alias_file=None, score_threshold=400)
        # MM 유전자 수동 추가
        for mm_gene in mm_genes:
            if mm_gene not in ppi_graph.nodes():
                ppi_graph.add_node(mm_gene)
        print(f"✓ Added {len(mm_genes)} MM genes to PPI")
    
    # 7. RWR + PPR 계산 (MM 허브 유전자로부터, 슈퍼 허브 제거)
    print("\n" + "="*70)
    print("Computing Network Propagation Scores (RWR + PPR)")
    print("="*70)
    
    # 7-1. RWR 계산 (직접 구현)
    rwr_scores = loader.compute_rwr_from_seeds(
        ppi_graph, mm_genes, 
        restart_prob=rwr_restart_prob,
        remove_super_hubs=remove_super_hubs
    )
    
    # 7-2. PPR 계산 (NetworkX 구현)
    ppr_scores = loader.compute_ppr_from_seeds(
        ppi_graph, mm_genes, 
        alpha=ppr_alpha,
        remove_super_hubs=remove_super_hubs
    )
    
    # 7-3. Seed 유전자의 RWR/PPR 점수를 0으로 설정 (bias 방지)
    # 목적: seed "주변" 유전자 발견, seed 자체는 이미 알려짐
    # Circular reasoning 방지 - 이미 아는 유전자가 높은 점수 받는 건 의미 없음
    print("\n[Seed Gene Exclusion from Ranking]")
    excluded_rwr = 0
    excluded_ppr = 0
    
    for seed in mm_genes:
        if seed in rwr_scores and rwr_scores[seed] > 0:
            rwr_scores[seed] = 0.0
            excluded_rwr += 1
        if seed in ppr_scores and ppr_scores[seed] > 0:
            ppr_scores[seed] = 0.0
            excluded_ppr += 1
    
    if excluded_rwr > 0:
        print(f"✓ RWR: Set {excluded_rwr} seed genes to 0 (avoid circular reasoning)")
    if excluded_ppr > 0:
        print(f"✓ PPR: Set {excluded_ppr} seed genes to 0 (avoid circular reasoning)")
    
    print(f"  → Seed genes excluded from ranking to prevent bias")
    print(f"  → Goal: Discover NOVEL genes around seeds, not re-discover seeds themselves")
    
    # Top 10 genes by RWR (seed 제외 후)
    print(f"\n[Top 10 genes by RWR score (seeds excluded)]")
    sorted_rwr = sorted(rwr_scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for gene, score in sorted_rwr:
        print(f"  {gene}: {score:.6f}")
    
    # Top 10 genes by PPR (seed 제외 후)
    print(f"\n[Top 10 genes by PPR score (seeds excluded)]")
    sorted_ppr = sorted(ppr_scores.items(), key=lambda x: x[1], reverse=True)[:10]
    for gene, score in sorted_ppr:
        print(f"  {gene}: {score:.6f}")
    
    # RWR과 PPR 점수 비교
    print(f"\n[Score Correlation Analysis]")
    common_genes = set(rwr_scores.keys()) & set(ppr_scores.keys())
    if len(common_genes) > 0:
        rwr_vals = np.array([rwr_scores[g] for g in common_genes])
        ppr_vals = np.array([ppr_scores[g] for g in common_genes])
        correlation = np.corrcoef(rwr_vals, ppr_vals)[0, 1]
        print(f"✓ RWR-PPR correlation: {correlation:.4f}")
        if correlation > 0.9:
            print(f"  → High agreement (>0.9): Both methods identify similar candidates")
        elif correlation > 0.7:
            print(f"  → Good agreement (0.7-0.9): Methods generally consistent")
        else:
            print(f"  → Moderate agreement (<0.7): Methods provide complementary information")
    
    # 8. 노드 인덱스 매핑
    node_indices_all = loader.create_gene_to_node_mapping(gene_list_all, ppi_graph)
    node_indices_train = loader.create_gene_to_node_mapping(gene_list_train, ppi_graph)
    
    # 9. 노드 특성 생성 (RWR + PPR 둘 다 포함!)
    node_features = loader.create_node_features_with_rwr_and_ppr(ppi_graph, rwr_scores, ppr_scores)
    
    # 10. 인접 행렬 생성
    adjacency_matrix = loader.create_adjacency_matrix(ppi_graph)
    
    # 11. Seed 유전자 여부 추가 (결과 분석용)
    # Note: Seed genes는 RWR/PPR 시작점이자 알려진 MM 연관 유전자
    # 이 표시는 결과 분석 단계에서 seed vs novel을 구분하기 위함
    genes_all = [genes[0] if isinstance(genes, list) else genes for genes in gene_list_all]
    df_all['is_seed_gene'] = [g in mm_genes for g in genes_all]
    
    print("\n" + "="*70)
    print("Data preparation completed!")
    print("="*70)
    print(f"[Training Set - Labeled only]")
    print(f"  X_train: {X_train.shape}")
    print(f"  y_train: {y_train.shape} (P={(y_train==1).sum()}, B={(y_train==0).sum()})")
    print(f"  node_indices_train: {node_indices_train.shape}")
    print(f"\n[Full Dataset - Including VUS]")
    print(f"  X_all: {X_all.shape}")
    print(f"  y_all: {y_all.shape} (P={(y_all==1).sum()}, B={(y_all==0).sum()}, VUS={(y_all==-1).sum()})")
    print(f"  node_indices_all: {node_indices_all.shape}")
    print(f"  Seed genes (RWR/PPR starts): {df_all['is_seed_gene'].sum()} variants")
    print(f"\n[Graph Data]")
    print(f"  node_features: {node_features.shape} (8D: centrality + RWR + PPR)")
    print(f"  adjacency_matrix: {adjacency_matrix.shape}")
    print(f"  Total nodes in PPI: {ppi_graph.number_of_nodes()}")
    print(f"  MM seed genes in PPI: {sum([g in ppi_graph.nodes() for g in mm_genes])}/{len(mm_genes)}")
    print("="*70)
    
    return X_train, y_train, X_all, y_all, node_features, adjacency_matrix, \
           node_indices_train, node_indices_all, df_all, ppi_graph, loader

# (Function) Modal_model

In [ ]:
class Modal1_Features_Classifier:
    """
    19개 원본 피처 + Classifier Probability
    UMAP은 시각화/분석용으로만 사용
    """
    def __init__(self, classifier_type='logistic', use_calibration=True, random_state=42):
        self.classifier_type = classifier_type
        self.use_calibration = use_calibration
        self.random_state = random_state
        
        # 분류기 선택
        if classifier_type == 'logistic':
            base_clf = LogisticRegression(random_state=random_state, max_iter=1000)
        elif classifier_type == 'svm':
            base_clf = SVC(kernel='rbf', probability=True, random_state=random_state)
        
        # Calibration 래퍼 (사용 가능한 경우만)
        if use_calibration and CALIBRATION_AVAILABLE:
            self.classifier = CalibratedClassifierCV(
                base_clf, 
                method='sigmoid',  # 🔥 isotonic → sigmoid (더 부드러운 보정)
                cv=3  # 3-fold CV로 보정
            )
            print(f"✓ Using calibrated {classifier_type} classifier (sigmoid method)")
        else:
            self.classifier = base_clf
            if use_calibration and not CALIBRATION_AVAILABLE:
                print(f"⚠ Calibration requested but not available (sklearn version issue)")
                print(f"  Using uncalibrated {classifier_type} classifier")
        
        self.scaler = StandardScaler()
        
        # UMAP은 시각화용 (Fusion에는 사용 안 함!)
        self.umap = UMAP(
            n_components=2,
            random_state=random_state,
            metric='euclidean',
            n_neighbors=30,
            min_dist=0.0,
            target_weight=0.5
        )
        self.umap_embedding = None  # 시각화용으로만 저장
        
    def fit(self, X_all, y_all, labeled_mask=None):
        """
        전체 데이터로 학습
        
        Parameters:
        -----------
        X_all: (n_all_samples, n_features) - VUS 포함 전체 입력 피처
        y_all: (n_all_samples,) - 라벨 (Pathogenic=1, Benign=0, VUS=-1)
        labeled_mask: (n_all_samples,) - True/False, 라벨이 있는 샘플만 True
        """
        # labeled_mask가 없으면 y != -1인 것들만 사용
        if labeled_mask is None:
            labeled_mask = (y_all != -1)
        
        labeled_count = labeled_mask.sum() if hasattr(labeled_mask, 'sum') else sum(labeled_mask)
        
        # 정규화: labeled 데이터로만 fit! (데이터 누수 방지)
        if labeled_count > 0:
            X_labeled = X_all[labeled_mask]
            self.scaler.fit(X_labeled)
            print(f"✓ Scaler fitted on {labeled_count} labeled samples only (no data leakage)")
        else:
            print("⚠ Warning: No labeled data. Fitting scaler on all data.")
            self.scaler.fit(X_all)
        
        # 전체 데이터 변환
        X_scaled = self.scaler.transform(X_all)
        
        # 1. UMAP 학습 (시각화용 - P/B만으로)
        if labeled_count > 0:
            X_labeled = X_scaled[labeled_mask]
            y_labeled = y_all[labeled_mask]
            
            print(f"[UMAP] Training on {labeled_count} labeled samples (for visualization)...")
            self.umap.fit(X_labeled, y=y_labeled)
            
            # 전체 데이터 변환 (시각화용)
            self.umap_embedding = self.umap.transform(X_scaled)
            print(f"✓ UMAP embedding created: {self.umap_embedding.shape}")
        else:
            print("Warning: No labeled data. Using unsupervised UMAP.")
            self.umap_embedding = self.umap.fit_transform(X_scaled)
        
        # 2. Classifier 학습 (19D + UMAP 2D = 21D 특성으로!)
        if labeled_count > 0:
            X_train = X_scaled[labeled_mask]
            y_train = y_all[labeled_mask]
            
            if len(np.unique(y_train)) >= 2:
                # UMAP 좌표도 포함 (transform과 동일하게!)
                umap_train = self.umap_embedding[labeled_mask]  # (n, 2)
                X_train_with_umap = np.hstack([X_train, umap_train])  # (n, 21)
                
                print(f"\n[Classifier] Training on {labeled_count} labeled samples (21D features: 19D + UMAP 2D)...")
                self.classifier.fit(X_train_with_umap, y_train)
                
                # 학습 정확도 확인
                train_acc = self.classifier.score(X_train_with_umap, y_train)
                print(f"✓ Classifier trained: P={sum(y_train==1)}, B={sum(y_train==0)}")
                print(f"  Training accuracy: {train_acc:.3f}")
            else:
                print(f"⚠ Warning: Only {len(np.unique(y_train))} class found. Cannot train classifier.")
        else:
            print("⚠ Warning: No labeled data for classifier training.")
        
        return self
    
    def transform(self, X):
        """
        Fusion용 특성 추출: 원본 19D + Probability
        
        핵심 로직:
        1. Classifier는 21D(19D + UMAP 2D)로 학습됨
        2. 따라서 predict_proba도 21D를 입력받아야 함
        3. 하지만 Fusion에는 20D(19D + Prob)만 전달
           → UMAP 정보는 Probability에 이미 반영됨
        
        Parameters:
        -----------
        X: (n_samples, 19) - 원본 특성
        
        Returns:
        --------
        summary_features: (n_samples, 20) = [19D features + P(Pathogenic)]
        """
        X_scaled = self.scaler.transform(X)
        
        # 1. UMAP 좌표 계산 (Classifier 입력용)
        try:
            umap_coords = self.umap.transform(X_scaled)  # (n, 2)
        except Exception as e:
            print(f"Warning: UMAP transform failed: {e}")
            print("Using zero coordinates")
            umap_coords = np.zeros((len(X), 2))
        
        # 2. Classifier 입력: 19D + UMAP 2D = 21D (학습 시와 동일!)
        X_for_clf = np.hstack([X_scaled, umap_coords])
        
        # 3. Probability 계산 (21D 입력!)
        try:
            proba = self.classifier.predict_proba(X_for_clf)[:, 1]  # P(Pathogenic)
            
            # 확률 clipping (포화 방지)
            proba = np.clip(proba, 0.01, 0.99)
            proba_values = proba.reshape(-1, 1)
        except (AttributeError, ValueError) as e:
            # 분류기 학습 안 된 경우 0.5로
            print(f"Warning: Classifier prediction failed: {e}")
            print("Using 0.5 as default probability.")
            proba_values = np.full((len(X), 1), 0.5)
        
        # 4. Fusion 입력: 원본 19D + Probability = 20D
        #    (UMAP은 Classifier 예측에만 사용, Fusion에는 안 들어감)
        summary_features = np.hstack([X_scaled, proba_values])
        # (n, 20) = [19D normalized features + P(Pathogenic)]
        
        return summary_features
    
    def fit_transform(self, X_all, y_all, labeled_mask=None):
        """
        fit과 transform을 한 번에
        """
        self.fit(X_all, y_all, labeled_mask)
        return self.transform(X_all)


# ==================== Modal 2: GCN for PPI Network ====================
class GCNLayer(nn.Module):
    """Graph Convolutional Layer"""
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, X, adj):
        """
        X: (n_nodes, in_features)
        adj: (n_nodes, n_nodes) - 정규화된 인접 행렬
        """
        support = self.linear(X)
        output = torch.matmul(adj, support)
        return output


class Modal2_GCN(nn.Module):
    """
    PPI 네트워크 기반 GCN 모델
    """
    def __init__(self, input_dim, hidden_dims=[64, 32], output_dim=16, dropout=0.5):
        super(Modal2_GCN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.dropout = dropout
        
        # GCN 레이어 구성
        layers = []
        dims = [input_dim] + hidden_dims + [output_dim]
        
        for i in range(len(dims) - 1):
            layers.append(GCNLayer(dims[i], dims[i+1]))
        
        self.layers = nn.ModuleList(layers)
        self.dropout_layer = nn.Dropout(dropout)
        
    def forward(self, X, adj):
        """
        X: (n_nodes, input_dim) - 노드 특성
        adj: (n_nodes, n_nodes) - 정규화된 인접 행렬
        """
        h = X
        for i, layer in enumerate(self.layers[:-1]):
            h = layer(h, adj)
            h = F.relu(h)
            h = self.dropout_layer(h)
        
        # 마지막 레이어 (활성화 없음)
        h = self.layers[-1](h, adj)
        
        return h
    
    @staticmethod
    def normalize_adjacency(adj):
        """
        인접 행렬 정규화: D^(-1/2) * A * D^(-1/2)
        """
        adj = adj + torch.eye(adj.size(0))  # Self-loop 추가
        degree = torch.sum(adj, dim=1)
        d_inv_sqrt = torch.pow(degree, -0.5)
        d_inv_sqrt[torch.isinf(d_inv_sqrt)] = 0.
        d_mat_inv_sqrt = torch.diag(d_inv_sqrt)
        normalized_adj = torch.matmul(torch.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)
        return normalized_adj


# ==================== Cross-Modal Attention ====================
class CrossModalAttention(nn.Module):
    """
    두 모달 간 상호 attention (샘플별 스칼라 게이트 방식)
    각 샘플이 독립적으로 처리되어 배치 간 누수 없음
    + 투영 레이어로 상대 모달 정보 직접 주입
    """
    def __init__(self, modal1_dim, modal2_dim, attention_dim=64):
        super(CrossModalAttention, self).__init__()
        
        # Modal 1 → Modal 2 게이트 (샘플별 스칼라)
        self.gate_m1_to_m2 = nn.Sequential(
            nn.Linear(modal1_dim + modal2_dim, attention_dim),
            nn.ReLU(),
            nn.Linear(attention_dim, 1),
            nn.Sigmoid()  # 0~1 게이트 값
        )
        
        # Modal 2 → Modal 1 게이트 (샘플별 스칼라)
        self.gate_m2_to_m1 = nn.Sequential(
            nn.Linear(modal1_dim + modal2_dim, attention_dim),
            nn.ReLU(),
            nn.Linear(attention_dim, 1),
            nn.Sigmoid()  # 0~1 게이트 값
        )
        
        # 차원 맞추기 위한 투영 레이어
        self.proj_m2_to_m1 = nn.Linear(modal2_dim, modal1_dim)  # 16D → 20D
        self.proj_m1_to_m2 = nn.Linear(modal1_dim, modal2_dim)  # 20D → 16D
        
    def forward(self, modal1_features, modal2_features):
        """
        modal1_features: (batch_size, modal1_dim) = 20D
        modal2_features: (batch_size, modal2_dim) = 16D
        
        Returns:
            attended_m1: gate * m1 + (1-gate) * Proj(m2)
            attended_m2: gate * m2 + (1-gate) * Proj(m1)
        """
        # 두 모달 결합
        combined = torch.cat([modal1_features, modal2_features], dim=1)
        
        # 샘플별 게이트 값 계산 (배치 간 독립)
        gate_12 = self.gate_m1_to_m2(combined)  # (B, 1) - Modal1이 Modal2 수용도
        gate_21 = self.gate_m2_to_m1(combined)  # (B, 1) - Modal2가 Modal1 수용도
        
        # 상대 모달 정보 투영
        m2_projected = self.proj_m2_to_m1(modal2_features)  # (B, 20) - Modal2를 Modal1 공간으로
        m1_projected = self.proj_m1_to_m2(modal1_features)  # (B, 16) - Modal1을 Modal2 공간으로
        
        # 게이트를 이용한 정보 융합
        # gate=1: 자기 자신만, gate=0: 상대방 정보만
        attended_m1 = gate_21 * modal1_features + (1 - gate_21) * m2_projected
        attended_m2 = gate_12 * modal2_features + (1 - gate_12) * m1_projected
        
        return attended_m1, attended_m2


# ==================== Cross-Modal Fusion MLP ====================
class CrossModalFusionMLP(nn.Module):
    """
    Cross-modal attention을 적용한 후 융합하여 최종 랭킹 스코어 출력
    """
    def __init__(self, modal1_dim, modal2_dim, attention_dim=64, hidden_dims=[128, 64, 32], dropout=0.3):
        super(CrossModalFusionMLP, self).__init__()
        
        # Cross-modal attention
        self.cross_attention = CrossModalAttention(modal1_dim, modal2_dim, attention_dim)
        
        # Fusion network
        # 원본 + attended 특성 모두 사용
        input_dim = (modal1_dim * 2) + (modal2_dim * 2)  # 원본 + attended
        layers = []
        dims = [input_dim] + hidden_dims + [1]
        
        for i in range(len(dims) - 1):
            layers.append(nn.Linear(dims[i], dims[i+1]))
            if i < len(dims) - 2:  # 마지막 레이어 제외
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(dropout))
        
        # 마지막 Sigmoid 제거 (BCEWithLogitsLoss 사용을 위해)
        # layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, modal1_features, modal2_features):
        """
        modal1_features: (batch_size, modal1_dim) = 20D
        modal2_features: (batch_size, modal2_dim) = 16D
        
        Returns: logits (not probabilities!)
        """
        # Cross-modal attention 적용
        attended_m1, attended_m2 = self.cross_attention(modal1_features, modal2_features)
        
        # 원본 특성 + attended 특성을 모두 결합
        fused = torch.cat([
            modal1_features, attended_m1,  # Modal 1: 20D + 20D
            modal2_features, attended_m2   # Modal 2: 16D + 16D
        ], dim=1)  # Total: 72D
        
        score = self.network(fused)
        return score.squeeze(-1)


# ==================== Late Fusion MLP (기존 버전 유지) ====================
class LateFusionMLP(nn.Module):
    """
    두 모달의 특성을 결합하여 최종 랭킹 스코어 출력 (단순 concatenation)
    """
    def __init__(self, modal1_dim, modal2_dim, hidden_dims=[128, 64, 32], dropout=0.3):
        super(LateFusionMLP, self).__init__()
        
        input_dim = modal1_dim + modal2_dim
        layers = []
        dims = [input_dim] + hidden_dims + [1]
        
        for i in range(len(dims) - 1):
            layers.append(nn.Linear(dims[i], dims[i+1]))
            if i < len(dims) - 2:  # 마지막 레이어 제외
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(dropout))
        
        # 마지막 Sigmoid 제거 (BCEWithLogitsLoss 사용)
        # layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, modal1_features, modal2_features):
        """
        modal1_features: (batch_size, modal1_dim)
        modal2_features: (batch_size, modal2_dim)
        
        Returns: logits (not probabilities!)
        """
        # Late fusion: 단순 concatenation
        fused = torch.cat([modal1_features, modal2_features], dim=1)
        score = self.network(fused)
        return score.squeeze(-1)


# ==================== 통합 멀티모달 모델 ====================
class MultimodalRankingModel:
    """
    전체 파이프라인을 통합한 멀티모달 랭킹 모델
    """
    def __init__(self, 
                 # Modal 1 파라미터
                 n_input_features=19,  # 입력 특성 개수
                 classifier_type='logistic',
                 use_calibration=True,  # Classifier 확률 보정 사용 여부
                 # Modal 2 파라미터 (GCN)
                 node_feature_dim=None,
                 gcn_hidden_dims=[64, 32],
                 gcn_output_dim=16,
                 # Fusion 파라미터
                 use_cross_attention=True,  # Cross-modal attention 사용 여부
                 attention_dim=64,
                 mlp_hidden_dims=[128, 64, 32],
                 dropout=0.3,
                 # 🔥 Score saturation 방지
                 temperature=1.0,  # Temperature scaling (>1이면 확률이 부드러워짐)
                 logits_clip=10.0,  # Logits clipping 범위
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        
        self.device = device
        self.use_cross_attention = use_cross_attention
        self.n_input_features = n_input_features
        self.temperature = temperature  # 🔥 Temperature scaling
        self.logits_clip = logits_clip  # 🔥 Logits clipping
        
        # Modal 1: 원본 특성 + Calibrated Classifier
        self.modal1 = Modal1_Features_Classifier(
            classifier_type=classifier_type,
            use_calibration=use_calibration
        )
        
        # Modal 2: GCN
        if node_feature_dim is not None:
            self.modal2_gcn = Modal2_GCN(
                input_dim=node_feature_dim,
                hidden_dims=gcn_hidden_dims,
                output_dim=gcn_output_dim,
                dropout=dropout
            ).to(device)
        else:
            self.modal2_gcn = None
        
        # Fusion MLP (Cross-attention 사용 여부에 따라 선택)
        modal1_dim = n_input_features + 1  # 19D + P(Pathogenic) = 20D
        modal2_dim = gcn_output_dim
        
        if use_cross_attention:
            self.fusion_mlp = CrossModalFusionMLP(
                modal1_dim=modal1_dim,
                modal2_dim=modal2_dim,
                attention_dim=attention_dim,
                hidden_dims=mlp_hidden_dims,
                dropout=dropout
            ).to(device)
        else:
            self.fusion_mlp = LateFusionMLP(
                modal1_dim=modal1_dim,
                modal2_dim=modal2_dim,
                hidden_dims=mlp_hidden_dims,
                dropout=dropout
            ).to(device)
        
        print(f"\n{'='*70}")
        print(f"Multimodal Ranking Model Initialized")
        print(f"{'='*70}")
        print(f"Modal 1: {n_input_features}D features → Classifier (21D: 19D+UMAP) → {modal1_dim}D")
        print(f"  → Classifier learns with: 19D original + 2D UMAP = 21D")
        print(f"  → Fusion receives: 19D original + 1D Probability = 20D")
        print(f"  → UMAP info is embedded in Probability!")
        print(f"Modal 2: GCN ({node_feature_dim}D → {gcn_output_dim}D)")
        print(f"Fusion: Cross-attention={'ON' if use_cross_attention else 'OFF'} (Fixed: No batch leakage)")
        print(f"Total fusion input: {modal1_dim}D + {modal2_dim}D = {modal1_dim + modal2_dim}D")
        print(f"Device: {device}")
        print(f"{'='*70}\n")
        
    def train_modal1(self, X_all, y_all, labeled_mask=None):
        """
        Modal 1 학습 (VUS 포함 전체 데이터 사용)
        
        Parameters:
        -----------
        X_all: (n_all, n_features) - VUS 포함 전체 변이 (19D)
        y_all: (n_all,) - 라벨 (VUS는 -1)
        labeled_mask: (n_all,) - 라벨 있는 샘플만 True (None이면 자동 생성)
        """
        print(f"\n{'='*70}")
        print(f"Training Modal 1 (Features + Classifier)")
        print(f"{'='*70}")
        self.modal1.fit(X_all, y_all, labeled_mask)
        print(f"{'='*70}\n")
        
    def train_fusion(self, 
                     X_tabular, 
                     node_features, 
                     adjacency_matrix, 
                     node_indices,
                     y_binary, 
                     epochs=100, 
                     lr=0.001, 
                     batch_size=32):
        """
        Fusion MLP 학습 (이진 분류)
        
        X_tabular: (n_samples, 19) - Modal 1 입력 (원본 특성)
        node_features: (n_nodes, feature_dim) - 그래프 노드 특성
        adjacency_matrix: (n_nodes, n_nodes) - 인접 행렬
        node_indices: (n_samples,) - 각 샘플에 해당하는 노드 인덱스
        y_binary: (n_samples,) - 이진 라벨 (0 or 1)
        """
        print(f"\n{'='*70}")
        print(f"Training Fusion Model")
        print(f"{'='*70}")
        
        # Modal 1 특성 추출 (19D + Probability)
        modal1_features = self.modal1.transform(X_tabular)  # (n, 20)
        modal1_features = torch.FloatTensor(modal1_features).to(self.device)
        
        print(f"Modal 1 features: {modal1_features.shape}")
        
        # Modal 2 특성 추출 (GCN)
        node_features_tensor = torch.FloatTensor(node_features).to(self.device)
        adj_tensor = torch.FloatTensor(adjacency_matrix).to(self.device)
        adj_normalized = Modal2_GCN.normalize_adjacency(adj_tensor)
        
        with torch.no_grad():
            all_node_embeddings = self.modal2_gcn(node_features_tensor, adj_normalized)
        
        modal2_features = all_node_embeddings[node_indices]
        
        print(f"Modal 2 features: {modal2_features.shape}")
        
        # 타겟 준비 (이진 라벨)
        y_tensor = torch.FloatTensor(y_binary).to(self.device)
        
        # 학습 (Weight decay 추가로 regularization 강화)
        optimizer = torch.optim.Adam(
            self.fusion_mlp.parameters(), 
            lr=lr, 
            weight_decay=1e-4  # 🔥 L2 regularization 추가
        )
        criterion = nn.BCEWithLogitsLoss()  # Logits 직접 사용 (수치 안정성 개선)
        
        dataset_size = len(y_binary)
        num_batches = (dataset_size + batch_size - 1) // batch_size  # ceil
        
        print(f"\nTraining for {epochs} epochs (batch_size={batch_size}, {num_batches} batches)...")
        
        for epoch in range(epochs):
            self.fusion_mlp.train()
            epoch_loss = 0
            
            # 미니배치 학습
            indices = torch.randperm(dataset_size)
            for i in range(0, dataset_size, batch_size):
                batch_indices = indices[i:i+batch_size]
                
                batch_modal1 = modal1_features[batch_indices]
                batch_modal2 = modal2_features[batch_indices]
                batch_y = y_tensor[batch_indices]
                
                optimizer.zero_grad()
                logits = self.fusion_mlp(batch_modal1, batch_modal2)  # logits (not proba)
                loss = criterion(logits, batch_y)
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
            
            if (epoch + 1) % 10 == 0:
                avg_loss = epoch_loss / num_batches  # 배치당 평균 손실
                print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
        
        print(f"{'='*70}\n")
    
    def predict(self, X_tabular, node_features, adjacency_matrix, node_indices):
        """
        최종 랭킹 스코어 예측 (Raw Logits)
        
        🎯 핵심 개념:
        - Logits = 모델의 "생 시그널" (raw confidence)
        - 높을수록 positive class에 가까움
        - Ranking에는 logits를 직접 사용! (상대적 순서만 중요)
        - 확률은 해석용으로만 사용 (predict_proba 메서드 참조)
        
        Returns:
            numpy.ndarray: Raw logits (ranking score)
        """
        self.fusion_mlp.eval()
        
        # Modal 1 특성 (19D + Probability)
        modal1_features = self.modal1.transform(X_tabular)
        modal1_features = torch.FloatTensor(modal1_features).to(self.device)
        
        # Modal 2 특성
        node_features_tensor = torch.FloatTensor(node_features).to(self.device)
        adj_tensor = torch.FloatTensor(adjacency_matrix).to(self.device)
        adj_normalized = Modal2_GCN.normalize_adjacency(adj_tensor)
        
        with torch.no_grad():
            all_node_embeddings = self.modal2_gcn(node_features_tensor, adj_normalized)
            modal2_features = all_node_embeddings[node_indices]
            
            # 🔥 Logits 예측 (이게 ranking score!)
            logits = self.fusion_mlp(modal1_features, modal2_features)
        
        # Logits를 그대로 반환 (ranking에 최적)
        return logits.cpu().numpy().flatten()
    
    def predict_proba(self, X_tabular, node_features, adjacency_matrix, node_indices):
        """
        확률 예측 (해석용)
        
        이 메서드는 사람이 이해하기 쉬운 0~1 확률로 변환합니다.
        실제 ranking에는 predict()의 logits를 사용하세요!
        
        Returns:
            numpy.ndarray: Sigmoid 확률 (0~1)
        """
        self.fusion_mlp.eval()
        
        # Modal 1 특성
        modal1_features = self.modal1.transform(X_tabular)
        modal1_features = torch.FloatTensor(modal1_features).to(self.device)
        
        # Modal 2 특성
        node_features_tensor = torch.FloatTensor(node_features).to(self.device)
        adj_tensor = torch.FloatTensor(adjacency_matrix).to(self.device)
        adj_normalized = Modal2_GCN.normalize_adjacency(adj_tensor)
        
        with torch.no_grad():
            all_node_embeddings = self.modal2_gcn(node_features_tensor, adj_normalized)
            modal2_features = all_node_embeddings[node_indices]
            
            # Logits 예측
            logits = self.fusion_mlp(modal1_features, modal2_features)
            
            # 🔥 Temperature scaling (확률을 부드럽게)
            scaled_logits = logits / self.temperature
            
            # Sigmoid로 확률 변환
            probabilities = torch.sigmoid(scaled_logits)
        
        return probabilities.cpu().numpy().flatten()


# ==================== Helper Functions ====================
def filter_superhubs(df, gene_column='Gene.refGene', verbose=True):
    """
    수퍼허브 유전자 필터링
    
    UBC, HSP90AA1, Histone, Ribosomal 단백질 등 
    일반적인 세포 기능 관련 수퍼허브 유전자 제거
    
    Parameters:
    -----------
    df: DataFrame
    gene_column: str, 유전자 이름 컬럼
    verbose: bool, 출력 여부
    
    Returns:
    --------
    filtered_df: 수퍼허브가 제거된 DataFrame
    """
    # 수퍼허브 유전자 리스트
    SUPER_HUB_GENES = [
        # UBC (Ubiquitin) 계열
        'UBC', 'UBB', 'UBA52', 'RPS27A',
        # HSP (Heat Shock Protein) 계열
        'HSP90AA1', 'HSP90AB1', 'HSPA8', 'HSPA1A', 'HSPA1B',
        'HSPA5', 'HSPA9', 'HSP90B1', 'HSPD1', 'HSPE1',
        # Actin/Tubulin
        'ACTB', 'ACTG1', 'TUBA1A', 'TUBA1B', 'TUBB',
    ]
    
    # 명시적 리스트
    mask_explicit = df[gene_column].isin(SUPER_HUB_GENES)
    
    # 패턴 매칭 (Histone, Ribosomal)
    mask_hist = df[gene_column].str.startswith('HIST', na=False)
    mask_rpl = df[gene_column].str.startswith('RPL', na=False)
    mask_rps = df[gene_column].str.startswith('RPS', na=False)
    
    # 전체 수퍼허브 마스크
    superhub_mask = mask_explicit | mask_hist | mask_rpl | mask_rps
    
    # 필터링
    filtered_df = df[~superhub_mask].copy()
    removed_count = superhub_mask.sum()
    
    if verbose and removed_count > 0:
        print(f"  🔥 Filtered out {removed_count} super-hub variants")
        removed_df = df[superhub_mask]
        removed_genes = removed_df[gene_column].value_counts().head(5)
        if len(removed_genes) > 0:
            print(f"     Top removed genes: {', '.join(removed_genes.index.tolist())}")
    
    return filtered_df

#  Main

In [ ]:
# ==================== Configuration ====================

# MM-related hub gene list (seed genes for the network modality)
mm_genes = [
    'CRBN', 'FGFR3', 'TP53', 'KRAS', 'TNFRSF17', 'NRAS', 'LIG4', 'BRAF', 'CXCR4', 
    'XPO1', 'CD38', 'RBX1', 'CUL4A', 'PSMB5', 'DDB1', 'BCL2', 'FDPS', 'TUBB4A'
] 
# open_target_0.6  # optional: genes from OpenTargets with score ≥ 0.6

# File path settings
annovar_file = r"clnvr140401_test_code.csv"
string_file = r"9606.protein.links.full.v12.0.txt"
alias_file = r"9606.protein.aliases.v12.0.txt"

CALIBRATION_AVAILABLE = True 

# Hyperparameters
EXONIC_ONLY = True          # use exonic variants only
USE_CLNSIG = True           # use ClinVar labels
RWR_RESTART_PROB = 0.15     # restart probability for RWR (Function)
PPR_ALPHA = 0.15            # teleportation probability for PPR (NetworkX)
STRING_THRESHOLD = 400      # minimum STRING combined score to keep an edge
REMOVE_SUPER_HUBS = True    # remove super-hub genes (e.g., UBC, histones) to improve disease specificity

# Model parameters
CLASSIFIER_TYPE = 'logistic'  # 'logistic' or 'svm'
USE_CROSS_ATTENTION = True    # enable cross-modal attention
GCN_HIDDEN = [64, 32]         # GCN hidden layer sizes
GCN_OUTPUT = 16               # GCN output dimension
MLP_HIDDEN = [128, 64, 32]    # fusion MLP hidden layer sizes
EPOCHS = 100                  # number of training epochs
LEARNING_RATE = 0.0001
BATCH_SIZE = 32

# Score saturation control
TEMPERATURE = 3.5             # temperature scaling (>1 makes probabilities smoother)
LOGITS_CLIP = 10.0            # logits clipping range (-10 to +10)

In [ ]:
"""
==================== MM Pathogenic Variant Prediction ====================
통합 실행 스크립트: 데이터 로딩 → 모델 학습 → 예측 → 분석
========================================================================== 
"""

print("""
╔═══════════════════════════════════════════════════════════════════╗
║     MM Pathogenic Variant Prediction - Integrated Pipeline       ║
╚═══════════════════════════════════════════════════════════════════╝
""")


# ==================== Step 1: Data Preparation ====================

print("\n" + "="*70)
print("STEP 1: Data Preparation")
print("="*70)

try:
    X_train, y_train, X_all, y_all, node_feat, adj, \
        node_idx_train, node_idx_all, df_all, ppi_graph, loader = prepare_variant_data(
        annovar_file=annovar_file,
        string_file=string_file,
        mm_genes=mm_genes,
        alias_file=alias_file,
        label_column='CLNSIG',
        gene_column='Gene.refGene',
        rwr_restart_prob=RWR_RESTART_PROB,  # RWR (직접 구현)
        ppr_alpha=PPR_ALPHA,                # PPR (NetworkX)
        use_prediction_labels=not USE_CLNSIG,
        exonic_only=EXONIC_ONLY,
        remove_super_hubs=REMOVE_SUPER_HUBS  # 🔥 슈퍼 허브 제거
    )
except Exception as e:
    print(f"\n❌ ERROR during data preparation: {e}")
    raise

# 데이터 검증
print(f"\n[Data Validation]")
print(f"✓ Training samples: {len(y_train)}")
print(f"✓ Total samples (including VUS): {len(y_all)}")
print(f"✓ VUS count: {(y_all == -1).sum()}")
print(f"✓ Input features: {X_all.shape[1]}D")
print(f"✓ Graph nodes: {node_feat.shape[0]}")
print(f"✓ Graph node features: {node_feat.shape[1]}D")

# 차원 검증
if X_all.shape[1] < 19:
    print(f"\n⚠ WARNING: Expected 19 features, got {X_all.shape[1]}")
    print("Some features may be missing from your Annovar file")
    print("Model will use available features")

if len(y_train) == 0:
    print("\n❌ ERROR: No labeled training data!")
    print("Please check:")
    print("1. CLNSIG column has valid values")
    print("2. Or set USE_CLNSIG=False to use prediction scores")
    raise ValueError("No training data available")

if len(np.unique(y_train)) < 2:
    print("\n❌ ERROR: Need at least 2 classes (P and B) for training!")
    raise ValueError("Insufficient class diversity")

print("\n✓ Data preparation successful!")


In [ ]:
# ==================== Step 2: Model Initialization ====================

print("\n" + "="*70)
print("STEP 2: Model Initialization")
print("="*70)

model = MultimodalRankingModel(
    n_input_features=X_all.shape[1],  # 19
    classifier_type=CLASSIFIER_TYPE,
    node_feature_dim=node_feat.shape[1],  # 7 (centrality + RWR)
    gcn_hidden_dims=GCN_HIDDEN,
    gcn_output_dim=GCN_OUTPUT,
    use_cross_attention=USE_CROSS_ATTENTION,
    attention_dim=64,
    mlp_hidden_dims=MLP_HIDDEN,
    dropout=0.3,
    temperature=TEMPERATURE,      # 🔥 Score saturation 방지
    logits_clip=LOGITS_CLIP       # 🔥 Logits clipping
)

# ==================== Step 3: Modal 1 Training ====================

print("\n" + "="*70)
print("STEP 3: Training Modal 1 (Features + Classifier)")
print("="*70)

# VUS 포함 전체 데이터로 학습 (UMAP은 시각화용)
labeled_mask = (y_all != -1)
model.train_modal1(X_all, y_all, labeled_mask=labeled_mask)

# 🔥 디버깅: Classifier 점수 확인
print("\n[DEBUG] Modal 1 Scores Check:")
if hasattr(model.modal1, 'rescaled_scores_'):
    scores = model.modal1.rescaled_scores_
    print(f"  Unique values: {len(np.unique(scores))}")
    print(f"  Range: [{scores.min():.6f}, {scores.max():.6f}]")
    print(f"  Sample (first 10): {scores[:10]}")
    df_all["modal1_score"] = scores
else:
    print("  ⚠️ rescaled_scores_ not found!")

# 🔥 디버깅: transform 테스트
print("\n[DEBUG] Transform Test:")
test_features = model.modal1.transform(X_all[:5])
print(f"  Output shape: {test_features.shape}")  # (5, 20) 이어야 함
print(f"  Sample output:")
print(test_features)

In [ ]:
# ==================== Step 4: Visualization (UMAP) ====================

print("\n" + "="*70)
print("STEP 4: UMAP Visualization")
print("="*70)

try:
    # UMAP 임베딩 가져오기
    umap_embedding = model.modal1.umap_embedding
    
    if umap_embedding is not None:
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Pathogenic (빨강)
        pathogenic_mask = (y_all == 1)
        ax.scatter(umap_embedding[pathogenic_mask, 0], 
                   umap_embedding[pathogenic_mask, 1],
                   c='red', label=f'Pathogenic (n={pathogenic_mask.sum()})', 
                   alpha=0.6, s=30, edgecolors='darkred', linewidth=0.5)
        
        # Benign (파랑)
        benign_mask = (y_all == 0)
        ax.scatter(umap_embedding[benign_mask, 0], 
                   umap_embedding[benign_mask, 1],
                   c='blue', label=f'Benign (n={benign_mask.sum()})', 
                   alpha=0.6, s=30, edgecolors='darkblue', linewidth=0.5)
        
        # VUS (회색)
        vus_mask = (y_all == -1)
        ax.scatter(umap_embedding[vus_mask, 0], 
                   umap_embedding[vus_mask, 1],
                   c='gray', label=f'VUS (n={vus_mask.sum()})', 
                   alpha=0.3, s=20, edgecolors='black', linewidth=0.3)
        
        ax.set_xlabel('UMAP Component 1', fontsize=12)
        ax.set_ylabel('UMAP Component 2', fontsize=12)
        ax.set_title('UMAP Embedding Space: Pathogenic vs Benign vs VUS', 
                     fontsize=14, fontweight='bold')
        ax.legend(loc='best', fontsize=10)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('umap_visualization.png', dpi=300, bbox_inches='tight')
        print("✓ UMAP visualization saved: umap_visualization.png")
        plt.close()
    else:
        print("⚠ UMAP embedding not available")
        
except Exception as e:
    print(f"⚠ Visualization failed: {e}")

In [ ]:
# ==================== Step 5: Fusion Model Training ====================

print("\n" + "="*70)
print("STEP 5: Training Fusion Model (Cross-modal Attention + MLP)")
print("="*70)

# 🔥 DEBUG: 학습 전 데이터 확인
print("\n[DEBUG] Fusion Training Data Check:")
print(f"X_train shape: {X_train.shape}")
print(f"y_train unique values: {np.unique(y_train)}")
print(f"y_train distribution: P={sum(y_train==1)}, B={sum(y_train==0)}")

model.train_fusion(
    X_tabular=X_train,
    node_features=node_feat,
    adjacency_matrix=adj,
    node_indices=node_idx_train.astype(np.int64),  # numpy array로 확실히 변환
    y_binary=y_train.astype(np.float32),  # float32로 변환
    epochs=EPOCHS,
    lr=LEARNING_RATE,
    batch_size=BATCH_SIZE
)

# 🔥 DEBUG: 학습 직후 예측 테스트
print("\n[DEBUG] Fusion Prediction Test (first 10):")
test_scores = model.predict(
    X_tabular=X_train[:10],
    node_features=node_feat,
    adjacency_matrix=adj,
    node_indices=node_idx_train[:10]
)
print(f"Scores: {test_scores.flatten()}")
print(f"Unique: {len(np.unique(test_scores))}")

# ==================== Step 6: Prediction ====================

print("\n" + "="*70)
print("STEP 6: Predicting Pathogenic Scores")
print("="*70)

print("\nPredicting all variants (including VUS)...")
ranking_scores = model.predict(X_all, node_feat, adj, node_idx_all)  # Raw logits (ranking용)
proba_scores = model.predict_proba(X_all, node_feat, adj, node_idx_all)  # 확률 (해석용)

# DataFrame에 결과 추가
df_all['ranking_score'] = ranking_scores  # Logits (ranking에 사용)
df_all['pathogenic_proba'] = proba_scores  # 확률 (사람이 읽기 쉬움)

print(f"✓ Prediction complete: {len(ranking_scores)} variants scored")
print(f"  - Ranking Score (logits) range: [{ranking_scores.min():.2f}, {ranking_scores.max():.2f}]")
print(f"  - Probability range: [{proba_scores.min():.4f}, {proba_scores.max():.4f}]")

# 기존 호환성을 위해 scores_all은 확률로 유지
scores_all = proba_scores

In [ ]:
# ==================== Step 7: Score Distribution Visualization ====================

print("\n" + "="*70)
print("STEP 7: Score Distribution Analysis")
print("="*70)

try:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Score 분포 (Histogram)
    ax1 = axes[0]
    ax1.hist(scores_all[y_all == 1], bins=50, alpha=0.6, color='red', 
             label='Pathogenic (True)', edgecolor='darkred')
    ax1.hist(scores_all[y_all == 0], bins=50, alpha=0.6, color='blue', 
             label='Benign (True)', edgecolor='darkblue')
    ax1.hist(scores_all[y_all == -1], bins=50, alpha=0.4, color='gray', 
             label='VUS (Unknown)', edgecolor='black')
    ax1.axvline(0.5, color='black', linestyle='--', linewidth=2, label='Decision Boundary')
    ax1.set_xlabel('Pathogenic Score', fontsize=12)
    ax1.set_ylabel('Count', fontsize=12)
    ax1.set_title('Pathogenic Score Distribution', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: VUS Score 분포만
    ax2 = axes[1]
    vus_scores = scores_all[y_all == -1]
    
    if len(vus_scores) > 0:
        # pd.Series로 변환하여 안전하고 편리한 통계 계산
        vus_scores = pd.Series(vus_scores, dtype='float64')
        
        # 통계량 미리 계산
        vus_median = vus_scores.median()
        vus_mean   = vus_scores.mean()
        vus_std    = vus_scores.std()
        p90        = vus_scores.quantile(0.90)
        p95        = vus_scores.quantile(0.95)
        
        # Histogram
        ax2.hist(vus_scores, bins=50, alpha=0.7, color='purple', edgecolor='darkviolet')
        
        # 통계선 그리기
        ax2.axvline(vus_mean, color='red', linestyle='--', linewidth=2, 
                    label=f'Mean: {vus_mean:.3f} (±{vus_std:.3f})')
        ax2.axvline(vus_median, color='orange', linestyle='--', linewidth=2, 
                    label=f'Median: {vus_median:.3f}')
        ax2.axvline(p90, color='darkgreen', linestyle=':', linewidth=2, 
                    label=f'P90: {p90:.3f}')
        ax2.axvline(p95, color='darkred', linestyle=':', linewidth=2, 
                    label=f'P95: {p95:.3f}')
        
        # 레이블 및 타이틀
        ax2.set_xlabel('Pathogenic Score', fontsize=12)
        ax2.set_ylabel('Count', fontsize=12)
        ax2.set_title(f'VUS Score Distribution (n={len(vus_scores)})', 
                      fontsize=14, fontweight='bold')
        ax2.legend(fontsize=9, loc='upper left')
        ax2.grid(True, alpha=0.3)
    else:
        ax2.text(0.5, 0.5, 'No VUS data', ha='center', va='center', 
                 fontsize=14, transform=ax2.transAxes)
    
    plt.tight_layout()
    plt.savefig('score_distribution.png', dpi=300, bbox_inches='tight')
    print("✓ Score distribution saved: score_distribution.png")
    plt.close()
    
except Exception as e:
    print(f"⚠ Visualization failed: {e}")

In [ ]:
# ==================== Step 8: Results Analysis ====================

print("\n" + "="*70)
print("STEP 8: Results Analysis")
print("="*70)

# VUS 필터링
vus_mask = (y_all == -1)

print(f"\n[Overall Statistics]")
print(f"Total variants: {len(df_all)}")
print(f"  Pathogenic: {(y_all == 1).sum()}")
print(f"  Benign: {(y_all == 0).sum()}")
print(f"  VUS: {vus_mask.sum()}")
print(f"\n[Seed Gene Statistics]")
seed_mask = df_all['is_seed_gene']
print(f"Total seed gene variants: {seed_mask.sum()}")
print(f"  Seed + Pathogenic: {(seed_mask & (y_all == 1)).sum()}")
print(f"  Seed + Benign: {(seed_mask & (y_all == 0)).sum()}")
print(f"  Seed + VUS: {(vus_mask & seed_mask).sum()}")

# Score 분포
print(f"\n[Score Distribution]")
print(f"Mean: {scores_all.mean():.3f}")
print(f"Std: {scores_all.std():.3f}")
print(f"Min: {scores_all.min():.3f}")
print(f"Max: {scores_all.max():.3f}")
print(f"\n[Risk Categories]")
print(f"High risk (>0.7): {(scores_all > 0.7).sum()}")
print(f"Medium risk (0.4-0.7): {((scores_all >= 0.4) & (scores_all <= 0.7)).sum()}")
print(f"Low risk (<0.4): {(scores_all < 0.4).sum()}")

# VUS 후보 추출 (Seed vs Novel 분리)
print(f"\n[VUS Candidate Analysis]")

# Seed 유전자 마스크
seed_gene_mask = df_all['Gene.refGene'].isin(mm_genes)

# Case 1: Seed 유전자의 VUS (Known genes - Clinical prioritization)
seed_vus_mask = vus_mask & seed_gene_mask
if seed_vus_mask.sum() > 0:
    print(f"\n[Seed Gene VUS - Known MM Genes]")
    seed_vus = df_all[seed_vus_mask].copy()
    seed_vus = filter_superhubs(seed_vus, verbose=True)
    seed_vus = seed_vus.sort_values('ranking_score', ascending=False)  #Logits로 정렬
    
    print(f"Total: {len(seed_vus)} variants in {seed_vus['Gene.refGene'].nunique()} seed genes")
    
    # 출력할 컬럼 선택 (ranking_score와 확률 모두 표시)
    output_cols = ['Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 
                   'CADD_phred', 'ranking_score', 'pathogenic_proba']
    available_cols = [col for col in output_cols if col in seed_vus.columns]
    
    print(f"\nTop 10 seed gene VUS (sorted by ranking_score):")
    print(seed_vus[available_cols].head(10).to_string(index=False))
    
    # 저장
    seed_vus.to_csv('seed_gene_vus_ranked.csv', index=False)
    print(f"\n✓ Seed gene VUS saved to: seed_gene_vus_ranked.csv")
    
    # 고위험 (확률 기준)
    high_risk_seed = seed_vus[seed_vus['pathogenic_proba'] > 0.7]
    if len(high_risk_seed) > 0:
        high_risk_seed.to_csv('seed_gene_vus_high_risk.csv', index=False)
        print(f"✓ High-risk seed VUS: {len(high_risk_seed)} variants (proba >0.7)")
else:
    print("\n⚠ No seed gene VUS found")

# Case 2: Non-seed 유전자의 VUS (Novel discoveries - Research prioritization)
novel_vus_mask = vus_mask & ~seed_gene_mask
if novel_vus_mask.sum() > 0:
    print(f"\n[Novel Gene VUS - New Discoveries]")
    novel_vus = df_all[novel_vus_mask].copy()
    novel_vus = filter_superhubs(novel_vus, verbose=True)  # 🔥 추가!
    novel_vus = novel_vus.sort_values('ranking_score', ascending=False)  # 🔥 Logits로 정렬!
    
    print(f"Total: {len(novel_vus)} variants in {novel_vus['Gene.refGene'].nunique()} genes")
    
    # 출력할 컬럼 선택 (ranking_score와 확률 모두 표시)
    output_cols = ['Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 
                   'CADD_phred', 'ranking_score', 'pathogenic_proba']
    available_cols = [col for col in output_cols if col in novel_vus.columns]
    
    print(f"\nTop 20 novel candidate VUS (sorted by ranking_score):")
    print(novel_vus[available_cols].head(20).to_string(index=False))
    
    # 저장
    novel_vus.to_csv('novel_candidate_vus.csv', index=False)
    print(f"\n✓ Novel candidate VUS saved to: novel_candidate_vus.csv")
    
    # 고위험 (확률 기준)
    high_risk_novel = novel_vus[novel_vus['pathogenic_proba'] > 0.7]
    if len(high_risk_novel) > 0:
        high_risk_novel.to_csv('novel_candidate_vus_high_risk.csv', index=False)
        print(f"✓ High-risk novel VUS: {len(high_risk_novel)} variants (proba >0.7)")
else:
    print("\n⚠ No novel candidate VUS found")

# 전체 결과 저장
all_results_file = 'all_variants_with_scores.csv'
df_all.to_csv(all_results_file, index=False)
print(f"\n✓ All variants with scores saved to: {all_results_file}")

In [ ]:
print("\n" + "="*70)
print("[DEBUG] Modal 1 vs Final Score Comparison")
print("="*70)

# 1. 먼저 0.982014 근처 값 찾기
print("\n[1] Searching for 0.982 scores...")
close_to_target = np.isclose(df_all['pathogenic_proba'], 0.982014, atol=0.001)
print(f"Found {close_to_target.sum()} variants with score ~0.982")

# 2. Novel gene 마스크 재생성
seed_gene_mask = df_all['Gene.refGene'].isin(mm_genes)
vus_mask = (y_all == -1)
novel_vus_mask = vus_mask & ~seed_gene_mask

print(f"\n[2] VUS breakdown:")
print(f"Total VUS: {vus_mask.sum()}")
print(f"Seed gene VUS: {(vus_mask & seed_gene_mask).sum()}")
print(f"Novel gene VUS: {novel_vus_mask.sum()}")

# 3. Novel gene 중 0.982 근처인 것들
novel_with_target_score = novel_vus_mask & close_to_target
print(f"\n[3] Novel genes with score ~0.982: {novel_with_target_score.sum()}")

# 4. 샘플 20개 확인
if novel_with_target_score.sum() > 0:
    sample_indices = df_all[novel_with_target_score].index[:20]
    
    print(f"\n[4] Detailed check (first 20):")
    print(f"{'Gene':<15} | {'Modal1_Prob':>12} | {'Final_Score':>12} | {'Node_Idx':>10}")
    print("-" * 60)
    
    for idx in sample_indices:
        gene = df_all.loc[idx, 'Gene.refGene']
        
        # Modal 1 출력
        modal1_out = model.modal1.transform(X_all[idx:idx+1])
        modal1_prob = modal1_out[0, -1]
        
        # Final score
        final_score = df_all.loc[idx, 'pathogenic_proba']
        
        # Node index
        node_idx = node_idx_all[idx]
        is_unknown = (node_idx == len(loader.gene_to_idx))
        
        print(f"{gene:<15} | {modal1_prob:>12.6f} | {final_score:>12.6f} | {node_idx:>10} {'(UNKNOWN)' if is_unknown else ''}")
    
    # 5. 통계
    print(f"\n[5] Statistics:")
    modal1_probs = []
    node_indices = []
    
    for idx in df_all[novel_with_target_score].index:
        modal1_out = model.modal1.transform(X_all[idx:idx+1])
        modal1_probs.append(modal1_out[0, -1])
        node_indices.append(node_idx_all[idx])
    
    modal1_probs = np.array(modal1_probs)
    node_indices = np.array(node_indices)
    unknown_count = (node_indices == len(loader.gene_to_idx)).sum()
    
    print(f"Modal 1 Probability range: [{modal1_probs.min():.6f}, {modal1_probs.max():.6f}]")
    print(f"Modal 1 Probability std: {modal1_probs.std():.6f}")
    print(f"UNKNOWN nodes: {unknown_count}/{len(node_indices)} ({unknown_count/len(node_indices)*100:.1f}%)")

In [ ]:
# ==================== Step 9: Performance Evaluation (라벨 있는 경우) ====================

print("\n" + "="*70)
print("STEP 9: Performance Evaluation")
print("="*70)

if labeled_mask.sum() > 0:
    from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
    from sklearn.metrics import classification_report, confusion_matrix, brier_score_loss
    
    y_true = y_all[labeled_mask]
    y_pred_proba = scores_all[labeled_mask]
    y_pred_binary = (y_pred_proba > 0.5).astype(int)
    
    print(f"\n[Classification Metrics]")
    print(f"ROC-AUC: {roc_auc_score(y_true, y_pred_proba):.3f}")
    print(f"PR-AUC: {average_precision_score(y_true, y_pred_proba):.3f}")
    print(f"Brier Score: {brier_score_loss(y_true, y_pred_proba):.3f} (lower is better)")
    print(f"Accuracy: {accuracy_score(y_true, y_pred_binary):.3f}")
    
    print(f"\n[Classification Report]")
    print(classification_report(y_true, y_pred_binary, 
                                 target_names=['Benign', 'Pathogenic']))
    
    print(f"\n[Confusion Matrix]")
    cm = confusion_matrix(y_true, y_pred_binary)
    print(f"              Predicted")
    print(f"              B    P")
    print(f"Actual  B   {cm[0,0]:4d} {cm[0,1]:4d}")
    print(f"        P   {cm[1,0]:4d} {cm[1,1]:4d}")
    
    # ROC Curve 시각화
    try:
        from sklearn.metrics import roc_curve
        
        fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
        
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='blue', lw=2, 
                 label=f'ROC curve (AUC = {roc_auc_score(y_true, y_pred_proba):.3f})')
        plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title('ROC Curve', fontsize=14, fontweight='bold')
        plt.legend(loc="lower right", fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
        print(f"\n✓ ROC curve saved: roc_curve.png")
        plt.close()
    except Exception as e:
        print(f"⚠ ROC curve plot failed: {e}")
else:
    print("⚠ No labeled data for evaluation")

In [ ]:
# ROC Curve 시각화 (계단 제거: PCHIP으로 '플롯만' 스무딩)
try:
    from sklearn.metrics import roc_curve, auc
    import numpy as np

    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)  # AUC는 원 점들로 계산(정직)

    # 단조 보존 스플라인 보간(계단 최소화). fpr 중복 제거 후 보간.
    from scipy.interpolate import PchipInterpolator
    ufpr, idx = np.unique(fpr, return_index=True)
    utpr = tpr[idx]
    x = np.linspace(0, 1, 1000)
    pchip = PchipInterpolator(ufpr, utpr)
    y = np.clip(pchip(x), 0, 1)

    plt.figure(figsize=(8, 6), dpi=180)
    # 부드러운 곡선
    plt.plot(x, y, linewidth=3, label=f"ROC (AUC = {roc_auc:.3f})")
    # 원래 계단 점들도 희미하게(선택: 안 예쁘면 지워도 됨)
    plt.step(fpr, tpr, where="post", linewidth=1, alpha=0.25)

    # 무작위 기준선
    plt.plot([0, 1], [0, 1], "--", linewidth=1.2, label="Random")

    # 보기 좋게 다듬기
    ax = plt.gca()
    ax.set_xlim(0, 1); ax.set_ylim(0, 1.02)
    ax.set_aspect("equal", adjustable="box")
    ax.grid(True, linestyle=":", linewidth=0.8, alpha=0.6)
    for s in ("top", "right"): ax.spines[s].set_visible(False)

    plt.title("ROC Curve", fontsize=18, weight="bold")
    plt.xlabel("False Positive Rate", fontsize=13)
    plt.ylabel("True Positive Rate", fontsize=13)
    plt.tick_params(axis="both", labelsize=11)
    plt.legend(loc="lower right", frameon=True, framealpha=0.9, fontsize=11)
    plt.tight_layout()

    # 벡터+비트맵 둘 다 저장
    plt.savefig("roc_curve.svg")                     # 논문용(벡터)
    plt.savefig("roc_curve.png", dpi=300, bbox_inches="tight")
    print("✓ ROC curve saved: roc_curve.svg, roc_curve.png")
    plt.close()

except Exception as e:
    print(f"⚠ ROC curve plot failed: {e}")


In [ ]:
# ==================== Final Summary ====================

print("\n" + "="*70)
print("PIPELINE COMPLETED SUCCESSFULLY!")
print("="*70)

print(f"\n[Generated Files]")
print(f"✓ umap_visualization.png - UMAP embedding space")
print(f"✓ score_distribution.png - Score distributions")
print(f"✓ roc_curve.png - ROC curve (if labeled data available)")
print(f"✓ all_variants_with_scores.csv - All variants with predictions")
print(f"\n[VUS Candidates - Separated by Type]")
if os.path.exists('seed_gene_vus_ranked.csv'):
    print(f"✓ seed_gene_vus_ranked.csv - Known MM genes VUS (clinical use)")
if os.path.exists('seed_gene_vus_high_risk.csv'):
    print(f"✓ seed_gene_vus_high_risk.csv - High-risk seed VUS (>0.7)")
if os.path.exists('novel_candidate_vus.csv'):
    print(f"✓ novel_candidate_vus.csv - Novel gene candidates (research use)")
if os.path.exists('novel_candidate_vus_high_risk.csv'):
    print(f"✓ novel_candidate_vus_high_risk.csv - High-risk novel VUS (>0.7)")

print(f"\n[Next Steps]")
print(f"1. Review UMAP visualization for data quality")
print(f"2. Seed gene VUS: Use for clinical decision-making")
print(f"3. Novel candidate VUS: Investigate for new MM associations")
print(f"4. Validate top predictions with literature/databases")
print(f"5. Consider functional validation of high-risk novel candidates")

print(f"\n{'='*70}")
print(f"Thank you for using MM Pathogenic Variant Prediction!")
print(f"{'='*70}\n")

In [ ]:
import re
import pandas as pd

def normalize_clnsig(x: str) -> str:
    s = (str(x) or "").strip()
    s = s.replace(" ", "")
    return s if s != "." else "."

def is_pathogenic_like(s: str) -> bool:
    s = normalize_clnsig(s)
    return s in {
        "Pathogenic","Likely_pathogenic","Pathogenic/Likely_pathogenic"
    }

def is_benign_like(s: str) -> bool:
    s = normalize_clnsig(s)
    return s in {
        "Benign","Likely_benign","Benign/Likely_benign"
    }

def is_vus_like(s: str) -> bool:
    s = normalize_clnsig(s)
    # VUS로 취급: '.', Uncertain*, Conflicting*, not_provided, risk_factor 등
    return (
        s == "." or
        s.startswith("Uncertain") or
        s.startswith("Conflicting") or
        s in {"not_provided","risk_factor","no_classification_for_the_single_variant"}
    )

# df_all: 점수 붙은 최종 데이터프레임 가정
df_all["CLNSIG_norm"] = df_all["CLNSIG"].map(normalize_clnsig)

# Top 20 선별 로직(현재 파이프라인과 동일하게 정렬)
top20 = df_all.sort_values("pathogenic_proba", ascending=False).head(20).copy()

# Top20 안에 라벨 섞임이 있는지 확인
leak_mask = top20["CLNSIG_norm"].apply(lambda s: is_pathogenic_like(s) or is_benign_like(s))
n_leak = int(leak_mask.sum())

print("\n[Leakage Check on Top 20]")
print(f"  Labeled (Pathogenic/Benign-like) in Top20: {n_leak}")
if n_leak > 0:
    print("  ⚠ Found labeled items inside Top-20 candidates. Showing them:")
    display_cols = ["Chr","Start","End","Ref","Alt","Gene.refGene","CLNSIG","pathogenic_proba"]
    print(top20.loc[leak_mask, display_cols].to_string(index=False))
else:
    print("  ✓ No labeled items inside Top-20.")

# ‘진짜 VUS 후보 Top20’ 재계산(라벨 엄격 제외)
mask_vus = df_all["CLNSIG_norm"].apply(is_vus_like)
top20_vus = (
    df_all.loc[mask_vus]
    .sort_values("pathogenic_proba", ascending=False)
    .head(20)
    .copy()
)

print("\n[Top 20 — VUS-only]")
display_cols = ["Chr","Start","End","Ref","Alt","Gene.refGene","CLNSIG","pathogenic_proba"]
print(top20_vus[display_cols].to_string(index=False))


In [ ]:
# 유전자별 최고 스코어
gene_top = (
    df_all
    .groupby("Gene.refGene", as_index=False)
    .agg(
        best_score=("pathogenic_proba", "max"),
        n_variants=("pathogenic_proba", "size")
    )
    .sort_values("best_score", ascending=False)
)

gene_top.head(50).to_csv("gene_level_vus_ranking.csv", index=False)
gene_top.head(20)


In [ ]:
# 1) Modal1 쪽에서 "정규화 + 확률"까지 포함된 20D 피처 뽑기
modal1_features = model.modal1.transform(X_all)
modal1_prob = modal1_features[:, -1]   # 마지막 컬럼이 P(Pathogenic)

# 2) 최종 점수 (fusion + GCN까지 지난 점수)
final_score = scores_all  # 또는 df_all["pathogenic_score"].values

# 3) 상관계수 + 차이 분포 보기
corr = np.corrcoef(modal1_prob, final_score)[0, 1]
print("corr(modal1, final) =", corr)

diff = np.abs(modal1_prob - final_score)
print("abs diff summary:")
print(pd.Series(diff).describe())


# IF you have Pretrained-Pkl file. Try  it.

In [ ]:
import pickle
import pandas as pd
import numpy as np

# 1) 모델 & 네트워크 로딩
with open('trained_model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('network_data.pkl', 'rb') as f:
    network_data = pickle.load(f)

ppi_graph = network_data['ppi_graph']
node_features = network_data['node_features']
adjacency_matrix = network_data['adjacency_matrix']

# 2) 테스트 파일 로딩
test_file = annovar_file
df_test = pd.read_csv(test_file)

# 필요하면 exonic/splicing 필터, synonymous 제거도 여기서 잠깐 해도 됨

# 3) feature & gene 매핑
loader = VariantDataLoader()

X_test, feat_names = loader.preprocess_features(df_test)
gene_list = loader.extract_gene_mapping(df_test, gene_column='Gene.refGene')
node_indices = loader.create_gene_to_node_mapping(gene_list, ppi_graph)

print("X_test:", X_test.shape)
print("node_indices:", node_indices.shape)

# 4) 멀티모달 모델로 예측
y_pred_proba = model.predict_proba(
    X_tabular=X_test,
    node_features=node_features,
    adjacency_matrix=adjacency_matrix,
    node_indices=node_indices
)

print("예측 확률 예시 (앞 5개):", y_pred_proba[:5])
print("평균:", np.mean(y_pred_proba), "최대:", np.max(y_pred_proba))

## Funtion for pretrained

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle

def filter_superhubs(df, gene_column='Gene.refGene', verbose=True):
    SUPER_HUB_GENES = [
        'UBC', 'UBB', 'UBA52', 'RPS27A',
        'HSP90AA1', 'HSP90AB1', 'HSPA8', 'HSPA1A', 'HSPA1B',
        'HSPA5', 'HSPA9', 'HSP90B1', 'HSPD1', 'HSPE1',
        'ACTB', 'ACTG1', 'TUBA1A', 'TUBA1B', 'TUBB',
    ]
    mask_explicit = df[gene_column].isin(SUPER_HUB_GENES)
    mask_hist = df[gene_column].str.startswith('HIST', na=False)
    mask_rpl = df[gene_column].str.startswith('RPL', na=False)
    mask_rps = df[gene_column].str.startswith('RPS', na=False)
    superhub_mask = mask_explicit | mask_hist | mask_rpl | mask_rps
    filtered_df = df[~superhub_mask].copy()
    removed_count = superhub_mask.sum()
    if verbose and removed_count > 0:
        print(f"  🔥 Filtered out {removed_count} super-hub variants")
        removed_genes = df[superhub_mask][gene_column].value_counts().head(5)
        if len(removed_genes) > 0:
            print(f"     Top removed genes: {', '.join(removed_genes.index.tolist())}")
    return filtered_df


def predict_new_variants(test_file, 
                         model_file='trained_model.pkl',
                         network_file='network_data.pkl',
                         output_dir='test_results',
                         verbose=True):
    os.makedirs(output_dir, exist_ok=True)
    
    if verbose:
        print("="*70)
        print("MM Pathogenic Variant Prediction - TEST MODE")
        print("="*70)
    
    # STEP 1: 모델 & 네트워크 로드
    if verbose:
        print("\n" + "="*70)
        print("STEP 1: Loading Trained Model & Network Data")
        print("="*70)
    
    if not os.path.exists(model_file):
        raise FileNotFoundError(f"Model file not found: {model_file}")
    if not os.path.exists(network_file):
        raise FileNotFoundError(f"Network file not found: {network_file}")
    
    if verbose:
        print(f"Loading model from: {model_file}")
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
    if verbose:
        print("✓ Model loaded")
    
    if verbose:
        print(f"\nLoading network data from: {network_file}")
    with open(network_file, 'rb') as f:
        network_data = pickle.load(f)
    
    ppi_graph = network_data['ppi_graph']
    mm_genes = network_data['mm_genes']
    node_feat = network_data['node_features']
    adj = network_data['adjacency_matrix']
    
    if verbose:
        print(f"✓ Network loaded:")
        print(f"  - PPI nodes: {ppi_graph.number_of_nodes()}")
        print(f"  - PPI edges: {ppi_graph.number_of_edges()}")
        print(f"  - MM seed genes: {len(mm_genes)}")
        print(f"  - Node features: {node_feat.shape}")
    
    # STEP 2: 테스트 데이터 로딩
    if verbose:
        print("\n" + "="*70)
        print("STEP 2: Loading Test Data")
        print("="*70)
        print(f"Test file: {test_file}")
    
    loader = VariantDataLoader()
    
    # 여기! load_annovar_data 사용
    df_test = loader.load_annovar_data(test_file)
    if verbose:
        print(f"✓ Loaded {len(df_test)} variants")
    
    # Exonic / Splicing 필터
    if 'Func.refGene' in df_test.columns:
        func_mask = (
            df_test['Func.refGene'].str.contains('exonic', case=False, na=False) |
            df_test['Func.refGene'].str.contains('splicing', case=False, na=False)
        )
        df_test = df_test[func_mask].copy()
        if verbose:
            print(f"✓ Filtered to exonic/splicing: {len(df_test)}")
    
    # Synonymous SNV 제외
    if 'ExonicFunc.refGene' in df_test.columns:
        syn_mask = df_test['ExonicFunc.refGene'].str.contains('synonymous SNV', case=False, na=False)
        df_test = df_test[~syn_mask].copy()
        if verbose:
            print(f"✓ Removed synonymous SNV: {len(df_test)}")
    
    # 19개 특성 추출
    X_test, feature_names = loader.preprocess_features(df_test)
    if verbose:
        print(f"✓ Extracted features: {X_test.shape}")
    
    # Gene → Node 매핑
    gene_list = loader.extract_gene_mapping(df_test, gene_column='Gene.refGene')
    node_idx_test = loader.create_gene_to_node_mapping(gene_list, ppi_graph)
    if verbose:
        print(f"✓ Mapped to network nodes (unique={len(set(node_idx_test))})")
    
    # STEP 3: 예측
    if verbose:
        print("\n" + "="*70)
        print("STEP 3: Making Predictions")
        print("="*70)
        print("Running model predictions...")
    
    ranking_scores = model.predict(X_test, node_feat, adj, node_idx_test)
    proba_scores   = model.predict_proba(X_test, node_feat, adj, node_idx_test)
    
    if verbose:
        print(f"✓ Predictions complete: {len(ranking_scores)} variants")
        print(f"  - Ranking Score range: [{ranking_scores.min():.2f}, {ranking_scores.max():.2f}]")
        print(f"  - Probability range: [{proba_scores.min():.4f}, {proba_scores.max():.4f}]")
    
    df_test['ranking_score'] = ranking_scores
    df_test['pathogenic_proba'] = proba_scores
    
    # … 이하 STEP 4~6 (seed/novel 나누고, superhub 필터, csv 저장, 그림 그리는 부분)
    # 은 네가 올린 코드 그대로 써도 됨 (그 부분은 인터페이스 안 깨짐)

    # 마지막에 stats, results dict 만드는 부분도 그대로 유지 가능
    # (원래 코드 복붙해서 쓰면 됨)

    # ==================== Step 4: Results Analysis ====================
    
    if verbose:
        print("\n" + "="*70)
        print("STEP 4: Results Analysis")
        print("="*70)
    
    # 통계
    stats = {
        'total_variants': len(df_test),
        'ranking_score_mean': ranking_scores.mean(),
        'ranking_score_std': ranking_scores.std(),
        'proba_mean': proba_scores.mean(),
        'proba_std': proba_scores.std(),
        'high_risk_count': (proba_scores > 0.7).sum(),
        'medium_risk_count': ((proba_scores >= 0.4) & (proba_scores <= 0.7)).sum(),
        'low_risk_count': (proba_scores < 0.4).sum(),
    }
    
    if verbose:
        print("\n[Score Distribution]")
        print(f"Ranking Score - Mean: {stats['ranking_score_mean']:.3f}, Std: {stats['ranking_score_std']:.3f}")
        print(f"Probability   - Mean: {stats['proba_mean']:.3f}, Std: {stats['proba_std']:.3f}")
        
        print(f"\n[Risk Categories by Probability]")
        print(f"High risk (>0.7):      {stats['high_risk_count']} variants ({stats['high_risk_count']/len(proba_scores)*100:.1f}%)")
        print(f"Medium risk (0.4-0.7): {stats['medium_risk_count']} variants")
        print(f"Low risk (<0.4):       {stats['low_risk_count']} variants")
    
    # Seed vs Novel 분리
    seed_mask = df_test['Gene.refGene'].isin(mm_genes)
    stats['seed_count'] = seed_mask.sum()
    stats['novel_count'] = (~seed_mask).sum()
    
    if verbose:
        print(f"\n[Seed vs Novel Genes]")
        print(f"Seed gene variants:  {stats['seed_count']} ({stats['seed_count']/len(df_test)*100:.1f}%)")
        print(f"Novel gene variants: {stats['novel_count']} ({stats['novel_count']/len(df_test)*100:.1f}%)")
    
    # ==================== Step 5: Top Candidates ====================
    
    if verbose:
        print("\n" + "="*70)
        print("STEP 5: Top Candidate Variants")
        print("="*70)
    
    results = {}
    
    # Seed gene variants
    if seed_mask.sum() > 0:
        if verbose:
            print(f"\n[Top 10 Seed Gene Variants]")
        seed_variants = df_test[seed_mask].copy()
        seed_variants = filter_superhubs(seed_variants, verbose=verbose)
        seed_variants = seed_variants.sort_values('ranking_score', ascending=False)
        
        output_cols = ['Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 
                       'CADD_phred', 'CLNSIG', 'ranking_score', 'pathogenic_proba', 'AF_patient']
        available_cols = [col for col in output_cols if col in seed_variants.columns]
        
        if verbose:
            print(seed_variants[available_cols].head(10).to_string(index=False))
        
        # 저장
        seed_file = os.path.join(output_dir, 'seed_gene_predictions.csv')
        seed_variants.to_csv(seed_file, index=False)
        if verbose:
            print(f"\n✓ Saved to: {seed_file}")
        
        results['seed_variants'] = seed_variants
    else:
        results['seed_variants'] = None
    
    # Novel gene variants
    if (~seed_mask).sum() > 0:
        if verbose:
            print(f"\n[Top 20 Novel Gene Variants]")
        novel_variants = df_test[~seed_mask].copy()
        novel_variants = filter_superhubs(novel_variants, verbose=verbose)
        novel_variants = novel_variants.sort_values('ranking_score', ascending=False)
        
        output_cols = ['Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 
                       'CADD_phred', 'CLNSIG', 'ranking_score', 'pathogenic_proba','AF_patient']
        available_cols = [col for col in output_cols if col in novel_variants.columns]
        
        if verbose:
            print(novel_variants[available_cols].head(20).to_string(index=False))
        
        # 저장
        novel_file = os.path.join(output_dir, 'novel_gene_predictions.csv')
        novel_variants.to_csv(novel_file, index=False)
        if verbose:
            print(f"\n✓ Saved to: {novel_file}")
        
        results['novel_variants'] = novel_variants
    else:
        results['novel_variants'] = None
    
    # 전체 결과 저장
    all_file = os.path.join(output_dir, 'all_predictions.csv')
    df_test_sorted = df_test.sort_values('ranking_score', ascending=False)
    df_test_sorted.to_csv(all_file, index=False)
    if verbose:
        print(f"\n✓ All predictions saved to: {all_file}")
    
    results['df_all'] = df_test_sorted
    
    # High-risk variants
    high_risk = df_test[df_test['pathogenic_proba'] > 0.7].copy()
    if len(high_risk) > 0:
        high_risk = filter_superhubs(high_risk, verbose=False)
        high_risk = high_risk.sort_values('ranking_score', ascending=False)
        high_risk_file = os.path.join(output_dir, 'high_risk_predictions.csv')
        high_risk.to_csv(high_risk_file, index=False)
        if verbose:
            print(f"✓ High-risk variants ({len(high_risk)}): {high_risk_file}")
        results['high_risk'] = high_risk
    else:
        results['high_risk'] = None
    
    # ==================== Step 6: Visualization ====================
    
    if verbose:
        print("\n" + "="*70)
        print("STEP 6: Creating Visualizations")
        print("="*70)
    
    try:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Score distribution
        ax1 = axes[0]
        ax1.hist(proba_scores, bins=50, alpha=0.7, color='purple', edgecolor='black')
        ax1.axvline(proba_scores.mean(), color='red', linestyle='--', 
                    label=f'Mean: {proba_scores.mean():.3f}')
        ax1.axvline(0.7, color='orange', linestyle=':', linewidth=2,
                    label='High-risk threshold (0.7)')
        ax1.set_xlabel('Pathogenic Probability', fontsize=12)
        ax1.set_ylabel('Count', fontsize=12)
        ax1.set_title('Test Set - Score Distribution', fontsize=14, fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Seed vs Novel
        ax2 = axes[1]
        seed_scores = proba_scores[seed_mask]
        novel_scores = proba_scores[~seed_mask]
        
        if len(seed_scores) > 0:
            ax2.hist(seed_scores, bins=30, alpha=0.6, color='blue', 
                     label=f'Seed genes (n={len(seed_scores)})', edgecolor='darkblue')
        if len(novel_scores) > 0:
            ax2.hist(novel_scores, bins=30, alpha=0.6, color='green',
                     label=f'Novel genes (n={len(novel_scores)})', edgecolor='darkgreen')
        
        ax2.set_xlabel('Pathogenic Probability', fontsize=12)
        ax2.set_ylabel('Count', fontsize=12)
        ax2.set_title('Seed vs Novel Genes', fontsize=14, fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plot_file = os.path.join(output_dir, 'test_predictions_distribution.png')
        plt.savefig(plot_file, dpi=300, bbox_inches='tight')
        if verbose:
            print(f"✓ Visualization saved: {plot_file}")
        plt.show()  # Jupyter에서 바로 표시
        
    except Exception as e:
        if verbose:
            print(f"⚠ Visualization failed: {e}")
    
    if verbose:
        print("\n" + "="*70)
        print("TEST PREDICTION COMPLETE!")
        print("="*70)
        print(f"\nResults saved to: {output_dir}/")
        print(f"  - all_predictions.csv")
        if results['seed_variants'] is not None:
            print(f"  - seed_gene_predictions.csv")
        if results['novel_variants'] is not None:
            print(f"  - novel_gene_predictions.csv")
        if results['high_risk'] is not None:
            print(f"  - high_risk_predictions.csv")
        print(f"  - test_predictions_distribution.png")
    
    results['stats'] = stats
    return results

In [ ]:
#Pretained-test #if have
test_file = r"clnvr140401_test_code.csv" #if have

predict_new_variants(test_file, 
                         model_file='trained_model.pkl', #if have
                         network_file='network_data.pkl', #if have
                         output_dir='test_results',
                         verbose=True)